In [18]:
# ============================================
# 실행 예시: PostgreSQL에서 데이터 로드 및 파싱 후 저장
# ============================================

# 설정
SOURCE_TABLE = "test_spec_01"  # 소스 테이블명 (여기를 수정하세요)
TARGET_TABLE = "test_spec_02"  # 타겟 테이블명 (여기를 수정하세요)
TRUNCATE_BEFORE_INSERT = True  # True: 기존 데이터 삭제 후 삽입, False: 기존 데이터 유지하고 추가

# 1. SQLAlchemy 엔진 생성
engine = get_sqlalchemy_engine()

if engine is None:
    print("❌ 엔진 생성 실패. 작업을 중단합니다.")
else:
    # 2. 데이터 로드
    print("\n" + "="*80)
    print("📥 데이터 로드 중...")
    print("="*80)
    df_filtered = load_data_from_table(engine, SOURCE_TABLE, allowed_disp_nm1)
    
    if df_filtered is not None and len(df_filtered) > 0:
        # 3. 데이터 파싱 (이전에 정의한 함수 사용)
        print("\n" + "="*80)
        print("🔄 데이터 파싱 중...")
        print("="*80)
        
        parsed_data = []
        parsed_data_needs_check = []
        unparsed_data = []
        
        for idx, row in df_filtered.iterrows():
            parsed_rows, success, needs_check = parse_dimensions_advanced(row)
            if success and parsed_rows:
                if needs_check:
                    parsed_data_needs_check.extend(parsed_rows)
                else:
                    parsed_data.extend(parsed_rows)
            else:
                unparsed_data.append(row)
        
        df_parsed = pd.DataFrame(parsed_data)
        df_parsed_needs_check = pd.DataFrame(parsed_data_needs_check)
        df_unparsed = pd.DataFrame(unparsed_data)
        
        # 파싱 통계 출력
        total_parsed = len(df_parsed) + len(df_parsed_needs_check)
        print(f"✅ 파싱 성공 (확실): {len(df_parsed)}개 행")
        print(f"⚠️  파싱 성공 (체크 필요): {len(df_parsed_needs_check)}개 행")
        print(f"❌ 파싱 실패: {len(df_unparsed)}개 행")
        print(f"📈 전체 대비 파싱률: {(total_parsed / len(df_filtered) * 100):.1f}%")
        
        # 4. PostgreSQL 테이블에 저장
        print("\n" + "="*80)
        print("💾 데이터 저장 중...")
        print("="*80)
        
        success = save_parsed_data_to_table(
            engine=engine,
            df_parsed=df_parsed,
            df_needs_check=df_parsed_needs_check,
            source_table_name=SOURCE_TABLE,
            target_table_name=TARGET_TABLE,
            truncate_before_insert=TRUNCATE_BEFORE_INSERT
        )
        
        if success:
            print("\n" + "="*80)
            print("✅ 전체 작업 완료!")
            print("="*80)
            print(f"📊 요약:")
            print(f"  - 소스 테이블: {SOURCE_TABLE}")
            print(f"  - 타겟 테이블: {TARGET_TABLE}")
            print(f"  - 저장된 데이터: {total_parsed}개 행")
            print(f"  - 기존 데이터 삭제: {'예' if TRUNCATE_BEFORE_INSERT else '아니오'}")
            print(f"  - 타겟 테이블 스키마: 소스 테이블 컬럼 + dimension_type, parsed_value, needs_check")
        else:
            print("\n❌ 데이터 저장 실패")
    else:
        print("\n❌ 데이터 로드 실패 또는 데이터 없음")

✅ SQLAlchemy 엔진 생성 성공

📥 데이터 로드 중...
✅ 테이블 'test_spec_01'에서 955개 행 로드 완료
✅ allowed_disp_nm1로 필터링: 625개 행

🔄 데이터 파싱 중...
✅ 파싱 성공 (확실): 80개 행
⚠️  파싱 성공 (체크 필요): 1769개 행
❌ 파싱 실패: 5개 행
📈 전체 대비 파싱률: 295.8%

💾 데이터 저장 중...
✅ 소스 테이블 'test_spec_01' 스키마 읽기 완료 (13개 컬럼)
✅ 테이블 'test_spec_02' 생성/확인 완료
   추가된 컬럼: dimension_type, parsed_value, needs_check
✅ 테이블 'test_spec_02'의 기존 데이터 삭제 완료
✅ 소스 테이블 'test_spec_01' 스키마 읽기 완료 (13개 컬럼)
✅ 테이블 'test_spec_02'에 1849개 행 저장 완료

✅ 전체 작업 완료!
📊 요약:
  - 소스 테이블: test_spec_01
  - 타겟 테이블: test_spec_02
  - 저장된 데이터: 1849개 행
  - 기존 데이터 삭제: 예
  - 타겟 테이블 스키마: 소스 테이블 컬럼 + dimension_type, parsed_value, needs_check


In [17]:
import pandas as pd
from sqlalchemy import create_engine, text, inspect
import os
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

def get_sqlalchemy_engine():
    """SQLAlchemy 엔진 생성"""
    try:
        connection_string = f"postgresql://{os.getenv('PG_USER')}:{os.getenv('PG_PASSWORD')}@{os.getenv('PG_HOST')}:{os.getenv('PG_PORT')}/{os.getenv('PG_DATABASE')}"
        engine = create_engine(connection_string)
        print(f"✅ SQLAlchemy 엔진 생성 성공")
        return engine
    except Exception as e:
        print(f"❌ SQLAlchemy 엔진 생성 실패: {e}")
        return None

def load_data_from_table(engine, table_name, allowed_disp_nm1):
    """
    PostgreSQL 테이블에서 데이터 로드
    
    Parameters:
    - engine: SQLAlchemy engine
    - table_name: 소스 테이블명
    - allowed_disp_nm1: 필터링할 disp_nm1 리스트
    
    Returns:
    - DataFrame
    """
    try:
        # 전체 데이터 로드
        query = f"SELECT * FROM {table_name}"
        df = pd.read_sql(query, engine)
        print(f"✅ 테이블 '{table_name}'에서 {len(df)}개 행 로드 완료")
        
        # allowed_disp_nm1로 필터링
        if allowed_disp_nm1 and len(allowed_disp_nm1) > 0:
            df_filtered = df[df['disp_nm1'].isin(allowed_disp_nm1)]
            print(f"✅ allowed_disp_nm1로 필터링: {len(df_filtered)}개 행")
        else:
            df_filtered = df
            print(f"⚠️  필터링 없이 전체 데이터 사용")
        
        return df_filtered
    except Exception as e:
        print(f"❌ 데이터 로드 실패: {e}")
        return None

def get_table_schema(engine, source_table_name):
    """
    소스 테이블의 스키마를 읽어옴
    
    Parameters:
    - engine: SQLAlchemy engine
    - source_table_name: 소스 테이블명
    
    Returns:
    - 컬럼 정보 딕셔너리 리스트
    """
    try:
        inspector = inspect(engine)
        columns = inspector.get_columns(source_table_name)
        print(f"✅ 소스 테이블 '{source_table_name}' 스키마 읽기 완료 ({len(columns)}개 컬럼)")
        return columns
    except Exception as e:
        print(f"❌ 스키마 읽기 실패: {e}")
        return None

def map_sqlalchemy_type_to_postgres(column_type):
    """
    SQLAlchemy 타입을 PostgreSQL 타입으로 변환
    """
    type_str = str(column_type)
    
    # 일반적인 타입 매핑
    if 'INTEGER' in type_str or 'BIGINT' in type_str or 'SMALLINT' in type_str:
        return 'INTEGER'
    elif 'SERIAL' in type_str or 'BIGSERIAL' in type_str:
        return 'SERIAL'
    elif 'VARCHAR' in type_str:
        # VARCHAR(길이) 추출
        return type_str.replace('VARCHAR', 'VARCHAR')
    elif 'TEXT' in type_str:
        return 'TEXT'
    elif 'BOOLEAN' in type_str or 'BOOL' in type_str:
        return 'BOOLEAN'
    elif 'TIMESTAMP' in type_str:
        return 'TIMESTAMP'
    elif 'DATE' in type_str:
        return 'DATE'
    elif 'NUMERIC' in type_str or 'DECIMAL' in type_str:
        return type_str.replace('NUMERIC', 'NUMERIC')
    elif 'FLOAT' in type_str or 'REAL' in type_str or 'DOUBLE' in type_str:
        return 'DOUBLE PRECISION'
    else:
        # 기본값
        return 'TEXT'

def create_parsed_table_from_source(engine, source_table_name, target_table_name):
    """
    소스 테이블의 스키마를 기반으로 파싱된 데이터를 저장할 테이블 생성
    dimension_type과 parsed_value 컬럼 추가
    
    Parameters:
    - engine: SQLAlchemy engine
    - source_table_name: 소스 테이블명
    - target_table_name: 생성할 테이블명
    """
    # 소스 테이블 스키마 읽기
    columns = get_table_schema(engine, source_table_name)
    if columns is None:
        return False
    
    # CREATE TABLE 쿼리 생성
    column_definitions = []
    
    for col in columns:
        col_name = col['name']
        col_type = map_sqlalchemy_type_to_postgres(col['type'])
        nullable = "NULL" if col['nullable'] else "NOT NULL"
        
        # PRIMARY KEY나 SERIAL 타입은 제거 (새 테이블에서는 id를 새로 만들 것)
        if col.get('autoincrement') or 'primary_key' in str(col).lower():
            continue
            
        column_definitions.append(f"{col_name} {col_type}")
    
    # dimension_type과 parsed_value 추가
    column_definitions.append("dimension_type TEXT")
    column_definitions.append("parsed_value NUMERIC")
    column_definitions.append("needs_check BOOLEAN")
    
    # CREATE TABLE 쿼리
    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {target_table_name} (
        id SERIAL PRIMARY KEY,
        {', '.join(column_definitions)},
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    """
    
    try:
        with engine.connect() as conn:
            conn.execute(text(create_table_query))
            conn.commit()
        print(f"✅ 테이블 '{target_table_name}' 생성/확인 완료")
        print(f"   추가된 컬럼: dimension_type, parsed_value, needs_check")
        return True
    except Exception as e:
        print(f"❌ 테이블 생성 실패: {e}")
        return False

def truncate_table(engine, table_name):
    """
    테이블의 기존 데이터 삭제
    
    Parameters:
    - engine: SQLAlchemy engine
    - table_name: 테이블명
    """
    try:
        with engine.connect() as conn:
            conn.execute(text(f"TRUNCATE TABLE {table_name} RESTART IDENTITY CASCADE"))
            conn.commit()
        print(f"✅ 테이블 '{table_name}'의 기존 데이터 삭제 완료")
        return True
    except Exception as e:
        print(f"❌ 데이터 삭제 실패: {e}")
        return False

def save_parsed_data_to_table(engine, df_parsed, df_needs_check, source_table_name, target_table_name, truncate_before_insert=False):
    """
    파싱된 데이터를 PostgreSQL 테이블에 저장
    소스 테이블의 모든 컬럼 + dimension_type, parsed_value, needs_check 저장
    
    Parameters:
    - engine: SQLAlchemy engine
    - df_parsed: 파싱 성공한 확실한 데이터
    - df_needs_check: 파싱 성공했지만 체크가 필요한 데이터
    - source_table_name: 소스 테이블명
    - target_table_name: 대상 테이블명
    - truncate_before_insert: True이면 기존 데이터 삭제
    
    Returns:
    - 성공 여부
    """
    try:
        # 테이블 생성
        if not create_parsed_table_from_source(engine, source_table_name, target_table_name):
            return False
        
        # 기존 데이터 삭제 옵션
        if truncate_before_insert:
            if not truncate_table(engine, target_table_name):
                return False
        
        # 두 DataFrame 합치기
        df_all = pd.DataFrame()
        
        if len(df_parsed) > 0:
            df_parsed_copy = df_parsed.copy()
            df_parsed_copy['needs_check'] = False
            df_all = pd.concat([df_all, df_parsed_copy], ignore_index=True)
        
        if len(df_needs_check) > 0:
            df_needs_check_copy = df_needs_check.copy()
            df_needs_check_copy['needs_check'] = True
            df_all = pd.concat([df_all, df_needs_check_copy], ignore_index=True)
        
        if len(df_all) == 0:
            print("⚠️  저장할 데이터가 없습니다.")
            return True
        
        # dimension_type과 parsed_value가 있는지 확인
        if 'dimension_type' not in df_all.columns or 'parsed_value' not in df_all.columns:
            print("❌ dimension_type 또는 parsed_value 컬럼이 없습니다.")
            return False
        
        # 소스 테이블의 컬럼 정보 가져오기
        source_columns = get_table_schema(engine, source_table_name)
        if source_columns is None:
            return False
        
        # 소스 컬럼명 리스트
        source_column_names = [col['name'] for col in source_columns if not col.get('autoincrement')]
        
        # 저장할 DataFrame 구성: 소스의 모든 컬럼 + dimension_type, parsed_value, needs_check
        df_to_save = pd.DataFrame()
        
        # 소스 테이블의 모든 컬럼 복사
        for col_name in source_column_names:
            if col_name in df_all.columns:
                df_to_save[col_name] = df_all[col_name]
        
        # 새로운 컬럼 추가
        df_to_save['dimension_type'] = df_all['dimension_type']
        df_to_save['parsed_value'] = df_all['parsed_value']
        df_to_save['needs_check'] = df_all['needs_check']
        
        # 데이터 저장
        df_to_save.to_sql(target_table_name, engine, if_exists='append', index=False)
        print(f"✅ 테이블 '{target_table_name}'에 {len(df_to_save)}개 행 저장 완료")
        return True
        
    except Exception as e:
        print(f"❌ 데이터 저장 실패: {e}")
        import traceback
        traceback.print_exc()
        return False

print("PostgreSQL 데이터 처리 함수 정의 완료")

PostgreSQL 데이터 처리 함수 정의 완료


In [10]:
import re

def identify_dimension_type(text):
    """
    텍스트에서 dimension 타입을 식별
    """
    text_lower = text.lower()
    
    # Depth 키워드
    if any(keyword in text_lower for keyword in ['두께', '깊이', 'd']):
        return 'depth'
    # Width 키워드
    elif any(keyword in text_lower for keyword in ['가로', '폭', 'w']):
        return 'width'
    # Height 키워드
    elif any(keyword in text_lower for keyword in ['세로', '높이', 'h']):
        return 'height'
    
    return None

def parse_dimensions_advanced(row):
    """
    disp_nm2에 따라 value를 파싱하는 함수 (확장 버전)
    """
    parsed_rows = []
    value = str(row['value'])
    disp_nm2 = str(row['disp_nm2'])
    needs_check = False  # 단위가 명확하지 않아 체크가 필요한 경우
    
    # 패턴 0: W숫자 x D숫자 x H숫자 형식 (예: "W269 x D375 x H269 mm")
    wdh_pattern = r'([WwHhDd])\s*(\d+(?:\.\d+)?)'
    wdh_matches = re.findall(wdh_pattern, value)
    
    if len(wdh_matches) >= 2:  # 최소 2개 이상의 dimension이 있는 경우
        base_row = row.to_dict()
        dimension_map = {'w': 'width', 'h': 'height', 'd': 'depth'}
        
        for dim_letter, num_val in wdh_matches:
            dim_type = dimension_map.get(dim_letter.lower())
            if dim_type:
                new_row = base_row.copy()
                new_row['dimension_type'] = dim_type
                new_row['parsed_value'] = float(num_val)
                new_row['needs_check'] = False
                parsed_rows.append(new_row)
        
        if parsed_rows:
            return parsed_rows, True, False
    
    # 패턴 1: value에 숫자(W), 숫자(H), 숫자(D)가 명시된 경우 (예: "276(W) x 327(H) x 293(D) mm", "178(W) x 68(H) x 72(D) mm")
    # 다양한 형식 지원: 숫자(W), 숫자mm(W), W:숫자 등
    whd_pattern = r'(\d+(?:\.\d+)?)\s*(?:mm)?\s*\(?\s*([WwHhDd])\s*\)?'
    whd_matches = re.findall(whd_pattern, value)
    
    if len(whd_matches) >= 2:  # 최소 2개 이상의 dimension이 있는 경우
        base_row = row.to_dict()
        dimension_map = {'w': 'width', 'h': 'height', 'd': 'depth'}
        
        for num_val, dim_letter in whd_matches:
            dim_type = dimension_map.get(dim_letter.lower())
            if dim_type:
                new_row = base_row.copy()
                new_row['dimension_type'] = dim_type
                new_row['parsed_value'] = float(num_val)
                new_row['needs_check'] = False
                parsed_rows.append(new_row)
        
        if parsed_rows:
            return parsed_rows, True, False
    
    # 패턴 2: "가로x높이x깊이" 텍스트가 있는 경우 (예: "820 x 56 x103.5 mm(가로x높이x깊이)")
    if '가로' in value and '높이' in value and '깊이' in value:
        nums = re.findall(r'(\d+(?:\.\d+)?)', value)
        if len(nums) >= 3:
            base_row = row.to_dict()
            
            # 가로 (width)
            row1 = base_row.copy()
            row1['dimension_type'] = 'width'
            row1['parsed_value'] = float(nums[0])
            row1['needs_check'] = False
            parsed_rows.append(row1)
            
            # 높이 (height)
            row2 = base_row.copy()
            row2['dimension_type'] = 'height'
            row2['parsed_value'] = float(nums[1])
            row2['needs_check'] = False
            parsed_rows.append(row2)
            
            # 깊이 (depth)
            row3 = base_row.copy()
            row3['dimension_type'] = 'depth'
            row3['parsed_value'] = float(nums[2])
            row3['needs_check'] = False
            parsed_rows.append(row3)
            
            return parsed_rows, True, False
    
    # 패턴 3: WxHxD 형식 (x로 구분, 단위 명시 없음) (예: "180 x 70 x 72 mm", "223 x 96.5 x 94 mm")
    wxhxd_match = re.search(r'(\d+(?:\.\d+)?)\s*[xX×]\s*(\d+(?:\.\d+)?)\s*[xX×]\s*(\d+(?:\.\d+)?)', value)
    if wxhxd_match:
        val1, val2, val3 = wxhxd_match.groups()
        base_row = row.to_dict()
        
        # 기본 가정: 가로 x 높이 x 깊이
        dimensions = [
            ('width', val1),
            ('height', val2),
            ('depth', val3)
        ]
        
        for dim_type, val in dimensions:
            new_row = base_row.copy()
            new_row['dimension_type'] = dim_type
            new_row['parsed_value'] = float(val)
            new_row['needs_check'] = True  # 단위가 명확하지 않음
            parsed_rows.append(new_row)
        
        return parsed_rows, True, True
    
    # 패턴 4: WxH 형식 (예: "500x600 mm")
    wxh_match = re.search(r'(\d+(?:\.\d+)?)\s*[xX×]\s*(\d+(?:\.\d+)?)', value)
    if wxh_match:
        val1, val2 = wxh_match.groups()
        base_row = row.to_dict()
        
        # 기본 가정: 가로 x 높이
        dimensions = [
            ('width', val1),
            ('height', val2)
        ]
        
        for dim_type, val in dimensions:
            new_row = base_row.copy()
            new_row['dimension_type'] = dim_type
            new_row['parsed_value'] = float(val)
            new_row['needs_check'] = True  # 단위가 명확하지 않음
            parsed_rows.append(new_row)
        
        return parsed_rows, True, True
    
    # 패턴 5: 단일 값 (disp_nm2에서 dimension 타입 식별)
    single_match = re.search(r'(\d+(?:\.\d+)?)', value)
    if single_match:
        dim_type = identify_dimension_type(disp_nm2)
        if dim_type:
            base_row = row.to_dict()
            base_row['dimension_type'] = dim_type
            base_row['parsed_value'] = float(single_match.group(1))
            base_row['needs_check'] = False
            parsed_rows.append(base_row)
            return parsed_rows, True, False
    
    return parsed_rows, False, False

# 필터링된 데이터의 모든 행에 대해 파싱 시도
parsed_data = []
parsed_data_needs_check = []
unparsed_data = []

print(f"전체 데이터: {len(df_filtered)}개 행")
print("파싱 중...\n")

for idx, row in df_filtered.iterrows():
    parsed_rows, success, needs_check = parse_dimensions_advanced(row)
    if success and parsed_rows:
        if needs_check:
            parsed_data_needs_check.extend(parsed_rows)
        else:
            parsed_data.extend(parsed_rows)
    else:
        unparsed_data.append(row)

# 새로운 DataFrame 생성
df_parsed = pd.DataFrame(parsed_data)
df_parsed_needs_check = pd.DataFrame(parsed_data_needs_check)
df_unparsed = pd.DataFrame(unparsed_data)

# 결과 통계
total_parsed = len(df_parsed) + len(df_parsed_needs_check)
total_original_rows = len(df_filtered)

print("=" * 80)
print("📊 파싱 결과 통계")
print("=" * 80)
print(f"✅ 파싱 성공 (확실): {len(df_parsed)}개 행")
print(f"⚠️  파싱 성공 (체크 필요): {len(df_parsed_needs_check)}개 행")
print(f"❌ 파싱 실패: {len(df_unparsed)}개 행")
print(f"📈 전체 대비 파싱률: {(total_parsed / total_original_rows * 100):.1f}%")
print("=" * 80)

# 파싱 성공한 데이터 출력 (확실한 것)
if len(df_parsed) > 0:
    print("\n✅ 파싱 성공 데이터 - 확실 (처음 20개):")
    print("-" * 80)
    display_cols = ['disp_nm1', 'disp_nm2', 'disp_nm3', 'disp_nm4', 'dimension_type', 'parsed_value', 'value']
    available_cols = [col for col in display_cols if col in df_parsed.columns]
    print(df_parsed[available_cols].head(20).to_string())
else:
    print("\n확실하게 파싱된 데이터가 없습니다.")

# 파싱 성공했지만 체크가 필요한 데이터 출력
if len(df_parsed_needs_check) > 0:
    print("\n\n⚠️  파싱 성공 데이터 - 체크 필요 (단위 명시 없음, 처음 20개):")
    print("-" * 80)
    display_cols = ['disp_nm1', 'disp_nm2', 'disp_nm3', 'disp_nm4', 'dimension_type', 'parsed_value', 'value']
    available_cols = [col for col in display_cols if col in df_parsed_needs_check.columns]
    print(df_parsed_needs_check[available_cols].head(20).to_string())
else:
    print("\n체크가 필요한 데이터가 없습니다.")

# 파싱 실패한 데이터 출력
if len(df_unparsed) > 0:
    print("\n\n❌ 파싱 실패 데이터 (처음 20개):")
    print("-" * 80)
    display_cols = ['disp_nm1', 'disp_nm2', 'disp_nm3', 'disp_nm4', 'value']
    available_cols = [col for col in display_cols if col in df_unparsed.columns]
    print(df_unparsed[available_cols].head(20).to_string())
    
    # 파싱 실패 패턴 분석
    print("\n\n❌ 파싱 실패 패턴 분석 (disp_nm2별 개수):")
    print("-" * 80)
    print(df_unparsed['disp_nm2'].value_counts().head(10))
else:
    print("\n\n모든 데이터가 성공적으로 파싱되었습니다!")

전체 데이터: 625개 행
파싱 중...

📊 파싱 결과 통계
✅ 파싱 성공 (확실): 80개 행
⚠️  파싱 성공 (체크 필요): 1769개 행
❌ 파싱 실패: 5개 행
📈 전체 대비 파싱률: 295.8%

✅ 파싱 성공 데이터 - 확실 (처음 20개):
--------------------------------------------------------------------------------
   disp_nm1         disp_nm2 dimension_type  parsed_value                                 value
0        규격               크기          width         276.0           276(W) x 327(H) x 293(D) mm
1        규격               크기         height         327.0           276(W) x 327(H) x 293(D) mm
2        규격               크기          depth         293.0           276(W) x 327(H) x 293(D) mm
3        규격               크기         height         216.0        216mm(H) x 790mm(W) x 287mm(D)
4        규격               크기          width         790.0        216mm(H) x 790mm(W) x 287mm(D)
5        규격               크기          depth         287.0        216mm(H) x 790mm(W) x 287mm(D)
6        규격               크기          width         820.0          820 x 56 x103.5 mm(가로x높이x깊이)
7      

In [ ]:
# 1. 필요한 라이브러리 임포트
import os
import psycopg2
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

print("라이브러리 임포트 완료")

# 2. PostgreSQL 연결 설정
def get_db_connection():
    """PostgreSQL 데이터베이스 연결"""
    try:
        conn = psycopg2.connect(
            host=os.getenv('PG_HOST'),
            port=os.getenv('PG_PORT'),
            database=os.getenv('PG_DATABASE'),
            user=os.getenv('PG_USER'),
            password=os.getenv('PG_PASSWORD')
        )
        print(f"✅ PostgreSQL 연결 성공: {os.getenv('PG_HOST')}")
        return conn
    except Exception as e:
        print(f"❌ PostgreSQL 연결 실패: {e}")
        return None

# 연결 테스트
conn = get_db_connection()
if conn:
    conn.close()

,mdl_code,goods_nm,disp_lv1,disp_lv2,disp_lv3,category_lv1,category_lv2,category_lv3,disp_nm1,disp_nm2,value,is_numeric,symbols,dimension_type,parsed_value
0,VS90F40CRG,Bespoke AI 제트 400W 침구브러시 패키지,리빙가전,청소기,BESPOKE AI 제트,진공청소기,VC STICK,Bespoke Jet AI,외관,"제품 크기(본체,WxHxD)",250x970x243 mm,NaN,mm,width,250.0
1,VS90F40CRG,Bespoke AI 제트 400W 침구브러시 패키지,리빙가전,청소기,BESPOKE AI 제트,진공청소기,VC STICK,Bespoke Jet AI,외관,"제품 크기(본체,WxHxD)",250x970x243 mm,NaN,mm,height,970.0
2,VS90F40CRG,Bespoke AI 제트 400W 침구브러시 패키지,리빙가전,청소기,BESPOKE AI 제트,진공청소기,VC STICK,Bespoke Jet AI,외관,"제품 크기(본체,WxHxD)",250x970x243 mm,NaN,mm,depth,243.0
3,VS90F40CNG,Bespoke AI 제트 400W 펫브러시 패키지,리빙가전,청소기,BESPOKE AI 제트,진공청소기,VC STICK,Bespoke Jet AI,외관,"제품 크기(본체,WxHxD)",250x970x243 mm,NaN,mm,width,250.0
4,VS90F40CNG,Bespoke AI 제트 400W 펫브러시 패키지,리빙가전,청소기,BESPOKE AI 제트,진공청소기,VC STICK,Bespoke Jet AI,외관,"제품 크기(본체,WxHxD)",250x970x243 mm,NaN,mm,height,970.0


In [4]:
# allowed_disp_nm1에 있는 값들만 필터링
df_filtered = df[df['disp_nm1'].isin(allowed_disp_nm1)]

# disp_nm2의 unique 값 출력
print("disp_nm2의 unique 값들:")
print(df_filtered['disp_nm2'].unique())
print(f"\n총 {df_filtered['disp_nm2'].nunique()}개의 unique 값")

disp_nm2의 unique 값들:
['크기' '크기(가로X깊이X높이)' '제품 크기(가로 × 높이 × 깊이)' '크기(세로x가로x두께, mm)'
 '크기(폭 × 높이 × 깊이)' '제품 크기(본체,WxHxD)' '제품크기 (가로x높이x깊이)' '제품크기'
 '제품크기 (WxDxH)' '제품크기 (가로x세로x높이)' '크기 (가로x높이x깊이)' '크기(가로x높이x깊이)'
 '크기(세로x가로x두께)' '크기(W×H×D)' '제품크기 (가로x높이x 깊이)' '크기(가로x깊이x높이)']

총 16개의 unique 값


In [2]:
import pandas as pd

# TSV 파일 읽기
df = pd.read_csv('/Users/toby/prog/kt/rubicon/data/kt_spec_validation_table_20251021.tsv', sep='\t')

# disp_nm1의 unique 리스트 출력
print("disp_nm1의 unique 값들:")
print(df['disp_nm1'].unique())
print(f"\n총 {df['disp_nm1'].nunique()}개의 unique 값")

disp_nm1의 unique 값들:
['규격' '사양' '외관 사양' '기본 사양' '외관' '팔레트' '기본사양' '기타제원' '마스터 박스' '유니트 박스'
 '본체치수' '주요사양' '하단패널' '일체형 청정스테이션' '일반사양']

총 15개의 unique 값


In [3]:
allowed_disp_nm1 = ['규격','사양','외관 사양','기본 사양','외관','기본사양','본체치수','주요사양','일반사양']

In [ ]:
# 1. 필요한 라이브러리 임포트
import os
import psycopg2
import pandas as pd
import numpy as np
from datetime import datetime
import json
from dotenv import load_dotenv
from openai import AzureOpenAI
from typing import Dict, List, Any
import warnings
warnings.filterwarnings('ignore')

# .env 파일 로드
load_dotenv()

print("라이브러리 임포트 완료")

In [ ]:
# 2. PostgreSQL 연결 설정
def get_db_connection():
    """PostgreSQL 데이터베이스 연결"""
    try:
        conn = psycopg2.connect(
            host=os.getenv('PG_HOST'),
            port=os.getenv('PG_PORT'),
            database=os.getenv('PG_DATABASE'),
            user=os.getenv('PG_USER'),
            password=os.getenv('PG_PASSWORD')
        )
        print(f"✅ PostgreSQL 연결 성공: {os.getenv('PG_HOST')}")
        return conn
    except Exception as e:
        print(f"❌ PostgreSQL 연결 실패: {e}")
        return None

# 연결 테스트
conn = get_db_connection()
if conn:
    conn.close()

In [ ]:
# 3. Azure OpenAI 클라이언트 설정
def get_openai_client():
    """Azure OpenAI 클라이언트 생성"""
    try:
        # 환경 변수 확인
        endpoint = os.getenv('ENDPOINT_URL')
        api_key = os.getenv('AZURE_OPENAI_API_KEY')
        api_version = os.getenv('AZURE_API_VERSION', '2024-02-01')  # 기본값 제공
        
        if not endpoint:
            print("❌ ENDPOINT_URL 환경 변수가 설정되지 않았습니다.")
            return None
        
        if not api_key:
            print("❌ AZURE_OPENAI_API_KEY 환경 변수가 설정되지 않았습니다.")
            return None
        
        # API 버전이 없으면 기본값 사용
        if not api_version:
            api_version = '2024-02-01'
            print(f"⚠️ AZURE_API_VERSION이 설정되지 않아 기본값 사용: {api_version}")
        
        print(f"📋 Azure OpenAI 설정:")
        print(f"  - Endpoint: {endpoint[:50]}...")
        print(f"  - API Version: {api_version}")
        print(f"  - Deployment: {os.getenv('DEPLOYMENT_NAME')}")
        
        client = AzureOpenAI(
            azure_endpoint=endpoint,
            api_key=api_key,
            api_version=api_version,
        )
        print(f"✅ Azure OpenAI 클라이언트 생성 성공")
        return client
    except Exception as e:
        print(f"❌ Azure OpenAI 클라이언트 생성 실패: {e}")
        return None

# 클라이언트 테스트
openai_client = get_openai_client()

# 간단한 테스트 호출
if openai_client:
    try:
        print("\n🧪 API 연결 테스트...")
        test_response = openai_client.chat.completions.create(
            model=os.getenv('DEPLOYMENT_NAME'),
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": "Say 'Hello' in one word."}
            ],
            max_tokens=10
        )
        if test_response and test_response.choices:
            print(f"✅ API 테스트 성공: '{test_response.choices[0].message.content}'")
        else:
            print("⚠️ API 응답이 비어있습니다.")
    except Exception as e:
        print(f"❌ API 테스트 실패: {e}")
        print(f"   상세 에러: {type(e).__name__}")
        if hasattr(e, '__dict__'):
            print(f"   에러 속성: {e.__dict__}")

In [ ]:
# 4. 테이블 스키마 정보 조회
def get_table_schema(table_name='test'):
    """테이블 스키마 정보 조회 (코멘트 포함)"""
    conn = get_db_connection()
    if not conn:
        return None
    
    try:
        # PostgreSQL에서 컬럼 정보와 코멘트를 함께 조회
        query = """
        SELECT 
            c.column_name,
            c.data_type,
            c.character_maximum_length,
            c.numeric_precision,
            c.numeric_scale,
            c.is_nullable,
            c.column_default,
            pgd.description as column_comment
        FROM information_schema.columns c
        LEFT JOIN pg_catalog.pg_statio_all_tables as st
            ON c.table_schema = st.schemaname 
            AND c.table_name = st.relname
        LEFT JOIN pg_catalog.pg_description pgd 
            ON pgd.objoid = st.relid 
            AND pgd.objsubid = c.ordinal_position
        WHERE c.table_schema = 'public' 
        AND c.table_name = %s
        ORDER BY c.ordinal_position;
        """
        
        df_schema = pd.read_sql_query(query, conn, params=(table_name,))
        print(f"✅ 테이블 '{table_name}' 스키마 조회 성공")
        print(f"   - 컬럼 수: {len(df_schema)}")
        
        # 코멘트가 있는 컬럼 수 확인
        comment_count = df_schema['column_comment'].notna().sum()
        print(f"   - 코멘트가 있는 컬럼: {comment_count}개")
        
        return df_schema
    
    except Exception as e:
        print(f"❌ 테이블 스키마 조회 실패: {e}")
        return None
    
    finally:
        conn.close()

# 스키마 조회
df_schema = get_table_schema(table_name="kt_merged_product_20250929")
if df_schema is not None:
    print("\n테이블 스키마 정보:")
    display(df_schema.head(10))

In [ ]:
# 5. kt_merged_product_20250929 테이블 데이터 조회
def get_product_data():
    """kt_merged_product_20250929 테이블에서 특정 컬럼만 조회"""
    conn = get_db_connection()
    if not conn:
        return None
    
    try:
        query = """
        SELECT 
            product_id,
            display_category_major,
            display_category_middle,
            display_category_minor,
            model_code,
            model_name,
            product_name,
            product_specification
        FROM kt_merged_product_20250929
        """
        
        df = pd.read_sql_query(query, conn)
        print(f"✅ 데이터 조회 성공: {len(df)}개 행")
        return df
    
    except Exception as e:
        print(f"❌ 데이터 조회 실패: {e}")
        return None
    
    finally:
        conn.close()

# 데이터 조회 및 표시
df_products = get_product_data()
if df_products is not None:
    print("\nkt_merged_product_20250929 테이블 데이터 (product_id, model_code, model_name, product_specification):")
    display(df_products)

In [ ]:
# 6. product_specification JSON 분석
def analyze_product_specifications():
    """product_specification JSON 필드를 상세 분석"""
    
    conn = get_db_connection()
    if not conn:
        return None
    
    try:
        # display_category_middle과 product_specification을 함께 조회
        query = """
        SELECT 
            display_category_middle,
            product_specification
        FROM kt_merged_product_20250929
        WHERE product_specification IS NOT NULL
        """
        
        df = pd.read_sql_query(query, conn)
        print(f"✅ 분석용 데이터 조회 성공: {len(df)}개 행\n")
        
        # 1. display_category_middle별 key-value 분석
        category_key_stats = {}
        all_keys = set()
        key_value_examples = {}
        
        for idx, row in df.iterrows():
            category = row['display_category_middle']
            spec = row['product_specification']
            
            if category not in category_key_stats:
                category_key_stats[category] = {
                    'keys': set(),
                    'key_counts': {},
                    'value_examples': {}
                }
            
            try:
                if isinstance(spec, str):
                    spec_dict = json.loads(spec)
                elif isinstance(spec, dict):
                    spec_dict = spec
                else:
                    continue
                    
                for key, value in spec_dict.items():
                    all_keys.add(key)
                    category_key_stats[category]['keys'].add(key)
                    
                    # key별 카운트
                    if key not in category_key_stats[category]['key_counts']:
                        category_key_stats[category]['key_counts'][key] = 0
                    category_key_stats[category]['key_counts'][key] += 1
                    
                    # value 예시 수집 (최대 3개)
                    if key not in category_key_stats[category]['value_examples']:
                        category_key_stats[category]['value_examples'][key] = set()
                    if len(category_key_stats[category]['value_examples'][key]) < 20:
                        if value and str(value).strip():
                            category_key_stats[category]['value_examples'][key].add(str(value)[:50])
                    
                    # 전체 key-value 예시
                    if key not in key_value_examples:
                        key_value_examples[key] = set()
                    # if len(key_value_examples[key]) < 5:
                    #     if value and str(value).strip():
                    #         key_value_examples[key].add(str(value)[:50])
                    if value and str(value).strip():
                        key_value_examples[key].add(str(value)[:50])
                            
            except Exception as e:
                continue
        
        # 2. display_category_middle별 주요 key 출력
        print("=" * 80)
        print("1. display_category_middle별 주요 Key와 Value 예시")
        print("=" * 80)
        
        for category in sorted(category_key_stats.keys())[:]:  
            stats = category_key_stats[category]
            total_products = sum(stats['key_counts'].values()) / len(stats['keys']) if stats['keys'] else 0
            
            print(f"\n📁 {category}")
            print(f"   총 제품 수: ~{int(total_products)}")
            print(f"   고유 key 수: {len(stats['keys'])}")
            
            top_keys = sorted(stats['key_counts'].items(), key=lambda x: x[1], reverse=True)[:]
            for key, count in top_keys:
                examples = list(stats['value_examples'].get(key, []))[:2]
                examples_str = ", ".join([f"'{ex}'" for ex in examples])
                print(f"   • {key}: {count}회 (예: {examples_str})")
        
        # 3. 특정 key가 어떤 display_category_middle에 속하는지 분석
        print("\n" + "=" * 80)
        print("2. 주요 Key별 소속 Category 분석")
        print("=" * 80)
        
        key_to_categories = {}
        for category, stats in category_key_stats.items():
            for key in stats['keys']:
                if key not in key_to_categories:
                    key_to_categories[key] = set()
                key_to_categories[key].add(category)
        
        # 여러 카테고리에 걸쳐 있는 key들
        cross_category_keys = {k: v for k, v in key_to_categories.items() if len(v) > 1}
        sorted_keys = sorted(cross_category_keys.items(), key=lambda x: len(x[1]), reverse=True)[:]
        
        print("\n📊 여러 카테고리에 걸쳐 있는 Key (상위 10개):")
        for key, categories in sorted_keys:
            print(f"   • {key}: {len(categories)}개 카테고리")
            print(f"     → {', '.join(list(categories)[:])}")
        
        # 단일 카테고리에만 있는 특화된 key들
        single_category_keys = {k: v for k, v in key_to_categories.items() if len(v) == 1}
        
        print("\n📌 특정 카테고리 전용 Key 예시:")
        category_specific = {}
        for key, categories in single_category_keys.items():
            cat = list(categories)[0]
            if cat not in category_specific:
                category_specific[cat] = []
            category_specific[cat].append(key)
        
        for cat, keys in list(category_specific.items())[:5]:
            print(f"   • {cat}: {', '.join(keys[:3])}")
        
        # 4. 전체 통계
        print("\n" + "=" * 80)
        print("3. 전체 JSON Key-Value 통계")
        print("=" * 80)
        
        # key별 전체 사용 횟수
        total_key_counts = {}
        for stats in category_key_stats.values():
            for key, count in stats['key_counts'].items():
                if key not in total_key_counts:
                    total_key_counts[key] = 0
                total_key_counts[key] += count
        
        print(f"\n📈 전체 통계:")
        print(f"   • 총 고유 key 수: {len(all_keys)}")
        print(f"   • 총 카테고리 수: {len(category_key_stats)}")
        print(f"   • 분석된 제품 수: {len(df)}")
        
        print(f"\n📊 가장 많이 사용되는 Key :")
        top_overall_keys = sorted(total_key_counts.items(), key=lambda x: x[1], reverse=True)[:]
        for key, count in top_overall_keys:
            percentage = (count / len(df)) * 100
            examples = list(key_value_examples.get(key, []))[:2]
            examples_str = ", ".join([f"'{ex}'" for ex in examples])
            print(f"   {key:30s} : {count:5d}회 ({percentage:5.1f}%) | 예: {examples_str}")
        
        # 5. 데이터 타입 분석
        print(f"\n📝 Value 데이터 타입 패턴:")
        type_patterns = {
            '숫자형': [],
            '불린형': [],
            '텍스트형': [],
            '단위포함': [],
            '리스트형': []
        }
        
        for key, examples in key_value_examples.items():
            sample = list(examples)[0] if examples else ""
            if sample:
                if sample.lower() in ['true', 'false', '예', '아니오']:
                    type_patterns['불린형'].append(key)
                elif any(unit in sample for unit in ['GB', 'MB', 'mm', 'cm', 'kg', 'W', 'Hz', 'mAh']):
                    type_patterns['단위포함'].append(key)
                elif sample.replace('.', '').replace('-', '').isdigit():
                    type_patterns['숫자형'].append(key)
                elif ',' in sample or '|' in sample:
                    type_patterns['리스트형'].append(key)
                else:
                    type_patterns['텍스트형'].append(key)
        
        for pattern_type, keys in type_patterns.items():
            if keys:
                print(f"   • {pattern_type}: {', '.join(keys[:5])}")
        
        return df
        
    except Exception as e:
        print(f"❌ 분석 실패: {e}")
        return None
    
    finally:
        conn.close()

# 분석 실행
df_spec_analysis = analyze_product_specifications()

In [ ]:
# 7. product_specification 고유 key 추출 및 유사 key 분석
def analyze_similar_keys():
    """product_specification의 모든 고유 key를 추출하고 유사한 key 쌍을 찾기"""
    
    conn = get_db_connection()
    if not conn:
        return None
    
    try:
        # 모든 product_specification 데이터 가져오기
        query = """
        SELECT product_specification
        FROM kt_merged_product_20250929
        WHERE product_specification IS NOT NULL
        """
        
        df = pd.read_sql_query(query, conn)
        print(f"✅ 데이터 조회 성공: {len(df)}개 행\n")
        
        # 모든 고유 key 추출
        all_keys = set()
        for spec in df['product_specification']:
            try:
                if isinstance(spec, str):
                    spec_dict = json.loads(spec)
                elif isinstance(spec, dict):
                    spec_dict = spec
                else:
                    continue
                    
                all_keys.update(spec_dict.keys())
            except:
                continue
        
        # 정렬된 key 리스트
        sorted_keys = sorted(list(all_keys))
        
        print("=" * 80)
        print(f"1. 전체 고유 Key 목록 (총 {len(sorted_keys)}개)")
        print("=" * 80)
        
        # 알파벳/한글별로 그룹화
        english_keys = [k for k in sorted_keys if k and k[0].isascii()]
        korean_keys = [k for k in sorted_keys if k and not k[0].isascii()]
        
        print("\n📌 영문 Key (한 줄씩):")
        print("-" * 40)
        for key in english_keys:
            print(key)
        
        print("\n📌 한글 Key (한 줄씩):")
        print("-" * 40)
        for key in korean_keys:
            print(key)
        
        # OpenAI를 사용한 유사 key 분석 (여러 번 호출)
        print("\n" + "=" * 80)
        print("2. OpenAI를 활용한 유사 Key 상세 분석")
        print("=" * 80)
        
        # OpenAI 클라이언트 가져오기
        openai_client = get_openai_client()
        
        all_similar_pairs = []
        
        if openai_client:
            try:
                # key를 배치로 나누어 분석 (한 번에 너무 많으면 API가 제대로 분석 못함)
                batch_size = 50
                for i in range(0, len(sorted_keys), batch_size):
                    batch_keys = sorted_keys[i:i+batch_size]
                    keys_str = ', '.join(batch_keys)
                    
                    print(f"\n🤖 배치 {i//batch_size + 1}/{(len(sorted_keys)-1)//batch_size + 1} 분석 중...")
                    
                    prompt = f"""다음은 제품 사양을 나타내는 JSON key 목록입니다:
{keys_str}

위 key들 중에서 의미가 유사하거나 중복되는 key 쌍들을 모두 찾아주세요.
예를 들어:
- 같은 속성을 다른 이름으로 표현한 경우
- 한글과 영문으로 중복된 경우
- 약어와 전체 이름이 함께 있는 경우
- 대소문자만 다른 경우
- 띄어쓰기나 언더스코어 차이만 있는 경우

가능한 모든 유사 쌍을 찾아서 JSON 형식으로 응답해주세요:
{{
  "similar_pairs": [
    {{"key1": "첫번째key", "key2": "두번째key", "reason": "유사한 이유"}},
    ...
  ]
}}
"""
                    
                    response = openai_client.chat.completions.create(
                        model=os.getenv('DEPLOYMENT_NAME'),
                        messages=[
                            {"role": "system", "content": "You are a data analyst expert in finding similar or duplicate fields in datasets. Find ALL possible similar pairs. Respond in Korean."},
                            {"role": "user", "content": prompt}
                        ],
                        max_tokens=3000,
                        temperature=0.2
                    )
                    
                    if response and response.choices:
                        result_text = response.choices[0].message.content
                        
                        # JSON 파싱 시도
                        try:
                            if '```json' in result_text:
                                json_str = result_text.split('```json')[1].split('```')[0]
                            elif '{' in result_text:
                                start = result_text.index('{')
                                end = result_text.rindex('}') + 1
                                json_str = result_text[start:end]
                            else:
                                json_str = result_text
                                
                            similar_data = json.loads(json_str)
                            
                            if 'similar_pairs' in similar_data:
                                all_similar_pairs.extend(similar_data['similar_pairs'])
                                
                        except json.JSONDecodeError:
                            pass
                
                # 중복 제거 및 출력
                print("\n📊 발견된 모든 유사 Key 쌍 (상세):")
                print("-" * 60)
                
                seen_pairs = set()
                for idx, pair in enumerate(all_similar_pairs, 1):
                    key1, key2 = pair.get('key1', ''), pair.get('key2', '')
                    pair_tuple = tuple(sorted([key1, key2]))
                    
                    if pair_tuple not in seen_pairs:
                        seen_pairs.add(pair_tuple)
                        print(f"\n{idx}. 유사 쌍:")
                        print(f"   Key 1: {key1}")
                        print(f"   Key 2: {key2}")
                        print(f"   이유: {pair.get('reason', '')}")
                        
            except Exception as e:
                print(f"❌ OpenAI API 호출 실패: {e}")
        else:
            print("⚠️ OpenAI 클라이언트를 사용할 수 없습니다.")
        
        # 상세한 규칙 기반 유사성 분석
        print("\n" + "=" * 80)
        print("3. 규칙 기반 유사 Key 상세 탐지")
        print("=" * 80)
        
        similar_groups = {
            '색상 관련': [],
            '크기/용량 관련': [],
            '무게 관련': [],
            '화면 관련': [],
            '배터리 관련': [],
            '메모리 관련': [],
            '카메라 관련': [],
            '네트워크/통신 관련': [],
            '프로세서/성능 관련': [],
            '오디오/사운드 관련': [],
            '연결/포트 관련': [],
            '전원 관련': [],
            '센서 관련': [],
            '보안 관련': [],
            '기타 기능': []
        }
        
        for key in sorted_keys:
            key_lower = key.lower()
            
            # 색상 관련
            if any(word in key_lower for word in ['color', '색상', '컬러', 'colour']):
                similar_groups['색상 관련'].append(key)
            
            # 크기/용량 관련
            if any(word in key_lower for word in ['size', '크기', '사이즈', '용량', 'capacity', 'dimension', '치수', 'volume']):
                similar_groups['크기/용량 관련'].append(key)
            
            # 무게 관련
            if any(word in key_lower for word in ['weight', '무게', '중량', 'mass']):
                similar_groups['무게 관련'].append(key)
            
            # 화면 관련
            if any(word in key_lower for word in ['display', '화면', '디스플레이', 'screen', 'panel', '패널', 'lcd', 'oled', 'resolution', '해상도']):
                similar_groups['화면 관련'].append(key)
            
            # 배터리 관련
            if any(word in key_lower for word in ['battery', '배터리', '전지', 'charge', '충전', 'power bank']):
                similar_groups['배터리 관련'].append(key)
            
            # 메모리 관련
            if any(word in key_lower for word in ['memory', '메모리', 'ram', 'storage', '저장', 'rom', 'ssd', 'hdd']):
                similar_groups['메모리 관련'].append(key)
            
            # 카메라 관련
            if any(word in key_lower for word in ['camera', '카메라', '촬영', '렌즈', 'lens', 'photo', '사진', 'video', '동영상']):
                similar_groups['카메라 관련'].append(key)
            
            # 네트워크/통신 관련
            if any(word in key_lower for word in ['network', '네트워크', 'wifi', 'wi-fi', 'bluetooth', '블루투스', 'lte', '5g', '4g', 'cellular']):
                similar_groups['네트워크/통신 관련'].append(key)
            
            # 프로세서/성능 관련
            if any(word in key_lower for word in ['processor', '프로세서', 'cpu', 'gpu', 'chip', '칩', 'performance', '성능']):
                similar_groups['프로세서/성능 관련'].append(key)
            
            # 오디오/사운드 관련
            if any(word in key_lower for word in ['audio', '오디오', 'sound', '사운드', 'speaker', '스피커', 'microphone', '마이크']):
                similar_groups['오디오/사운드 관련'].append(key)
            
            # 연결/포트 관련
            if any(word in key_lower for word in ['port', '포트', 'usb', 'hdmi', 'connector', '연결', 'jack', '잭']):
                similar_groups['연결/포트 관련'].append(key)
            
            # 전원 관련
            if any(word in key_lower for word in ['power', '전원', 'voltage', '전압', 'adapter', '어댑터', 'charger']):
                similar_groups['전원 관련'].append(key)
            
            # 센서 관련
            if any(word in key_lower for word in ['sensor', '센서', 'gyro', '자이로', 'accelerometer', '가속도계']):
                similar_groups['센서 관련'].append(key)
            
            # 보안 관련
            if any(word in key_lower for word in ['security', '보안', 'fingerprint', '지문', 'face', '얼굴', 'lock', '잠금']):
                similar_groups['보안 관련'].append(key)
        
        print("\n📌 주제별 유사 Key 그룹 (전체 목록):")
        print("-" * 60)
        
        for group_name, keys in similar_groups.items():
            if keys:
                print(f"\n【{group_name}】 총 {len(keys)}개")
                print("-" * 40)
                for key in sorted(keys):
                    print(key)
        
        # 대소문자/언더스코어 차이만 있는 key 찾기
        print("\n" + "=" * 80)
        print("4. 형식만 다른 중복 Key 탐지")
        print("=" * 80)
        
        normalized_keys = {}
        for key in sorted_keys:
            # 정규화: 소문자로 변환, 언더스코어/하이픈/공백 제거
            normalized = key.lower().replace('_', '').replace('-', '').replace(' ', '')
            if normalized not in normalized_keys:
                normalized_keys[normalized] = []
            normalized_keys[normalized].append(key)
        
        print("\n📌 형식만 다른 중복 가능 Key:")
        print("-" * 40)
        
        duplicate_count = 0
        for normalized, original_keys in normalized_keys.items():
            if len(original_keys) > 1:
                duplicate_count += 1
                print(f"\n중복 그룹 {duplicate_count}:")
                for key in original_keys:
                    print(f"  - {key}")
        
        if duplicate_count == 0:
            print("형식만 다른 중복 key가 발견되지 않았습니다.")
        
        return sorted_keys
        
    except Exception as e:
        print(f"❌ 분석 실패: {e}")
        return None
    
    finally:
        conn.close()

# 분석 실행
unique_keys = analyze_similar_keys()

In [ ]:
df_products.head(10)

In [ ]:
# 10. product_specification 계층 구조 분석
import json
import pandas as pd

def get_json_depth(obj, current_depth=0):
    """JSON 객체의 최대 깊이를 재귀적으로 계산"""
    if not isinstance(obj, (dict, list)):
        return current_depth
    
    if isinstance(obj, dict):
        if not obj:
            return current_depth
        return max(get_json_depth(v, current_depth + 1) for v in obj.values())
    
    if isinstance(obj, list):
        if not obj:
            return current_depth
        return max(get_json_depth(item, current_depth) for item in obj)

def flatten_json_hierarchy(json_obj, max_depth=5):
    """JSON을 계층별 key-value 쌍으로 변환"""
    result = {}
    
    # 각 계층 초기화
    for i in range(1, max_depth + 1):
        result[f'level_{i}_key'] = None
        result[f'level_{i}_value'] = None
    
    def traverse(obj, path=[], depth=1):
        if depth > max_depth:
            return
            
        if isinstance(obj, dict):
            for key, value in obj.items():
                if depth == 1:
                    result[f'level_{depth}_key'] = key
                    
                    # value가 단순 타입인 경우
                    if not isinstance(value, (dict, list)):
                        result[f'level_{depth}_value'] = str(value) if value is not None else None
                    # value가 dict인 경우 하위 계층 탐색
                    elif isinstance(value, dict):
                        result[f'level_{depth}_value'] = 'object'
                        traverse(value, path + [key], depth + 1)
                    # value가 list인 경우
                    elif isinstance(value, list):
                        result[f'level_{depth}_value'] = f'array[{len(value)}]'
                        # 첫 번째 요소만 탐색
                        if value and isinstance(value[0], dict):
                            traverse(value[0], path + [key], depth + 1)
    
    if isinstance(json_obj, dict):
        traverse(json_obj)
    
    return result

# DB에서 데이터 로드
conn = get_db_connection()
if conn:
    try:
        # product_specification이 not null인 데이터만 조회
        query = """
        SELECT 
            product_id,
            display_category_major,
            display_category_middle,
            display_category_minor,
            product_specification
        FROM kt_merged_product_20250929
        WHERE product_specification IS NOT NULL
        LIMIT 10000
        """
        
        df_products = pd.read_sql_query(query, conn)
        print(f"📊 데이터 로드 완료: {len(df_products)}개 행")
        
        # JSON 파싱 및 최대 깊이 계산
        max_depth = 0
        valid_specs = []
        
        for idx, row in df_products.iterrows():
            try:
                spec_str = row['product_specification']
                if spec_str:
                    # JSON 파싱
                    if isinstance(spec_str, str):
                        spec_json = json.loads(spec_str)
                    else:
                        spec_json = spec_str
                    
                    # 깊이 계산
                    depth = get_json_depth(spec_json)
                    max_depth = max(max_depth, depth)
                    
                    valid_specs.append((idx, spec_json))
            except Exception as e:
                print(f"  ⚠️ 행 {idx} JSON 파싱 실패: {e}")
                continue
        
        print(f"\n📏 최대 JSON 계층 깊이: {max_depth}")
        print(f"✅ 유효한 JSON 데이터: {len(valid_specs)}개")
        
        # 계층별로 데이터프레임 생성
        rows_data = []
        
        for idx, spec_json in valid_specs[:1000]:  # 시범적으로 1000개만 처리
            row = df_products.iloc[idx]
            
            # 기본 정보
            row_dict = {
                'product_id': row['product_id'],
                'display_category_major': row['display_category_major'],
                'display_category_middle': row['display_category_middle'],
                'display_category_minor': row['display_category_minor']
            }
            
            # JSON 계층 정보 추가
            hierarchy = flatten_json_hierarchy(spec_json, max_depth)
            row_dict.update(hierarchy)
            
            rows_data.append(row_dict)
        
        # 최종 데이터프레임 생성
        df_hierarchy = pd.DataFrame(rows_data)
        
        print(f"\n📋 계층 구조 데이터프레임 생성 완료")
        print(f"   - 행 수: {len(df_hierarchy)}")
        print(f"   - 컬럼 수: {len(df_hierarchy.columns)}")
        print(f"\n컬럼 목록:")
        for col in df_hierarchy.columns:
            print(f"   - {col}")
        
        # 첫 5개 행 출력
        print("\n🔍 데이터 샘플 (첫 5개 행):")
        display(df_hierarchy.tail(10))
        
        # 각 계층별 고유 key 분석
        print("\n📊 각 계층별 고유 key 분석:")
        for i in range(1, max_depth + 1):
            key_col = f'level_{i}_key'
            if key_col in df_hierarchy.columns:
                unique_keys = df_hierarchy[key_col].dropna().unique()
                print(f"\n   Level {i}:")
                print(f"   - 고유 key 개수: {len(unique_keys)}")
                if len(unique_keys) <= 20:
                    print(f"   - Keys: {sorted(unique_keys)}")
                else:
                    print(f"   - Top 10 Keys: {sorted(unique_keys)[:10]}")
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        import traceback
        traceback.print_exc()
    finally:
        conn.close()
else:
    print("❌ DB 연결 실패")

In [ ]:
# 11. product_specification 계층 구조 상세 분석 (개선 버전)
import json
import pandas as pd
from typing import Dict, List, Any, Tuple, Optional

def extract_nested_values(obj, parent_keys=None, results=None):
    """
    중첩된 JSON 객체에서 모든 leaf 값들을 추출
    parent_keys: 상위 key들의 리스트
    results: 결과를 저장할 리스트
    """
    if parent_keys is None:
        parent_keys = []
    if results is None:
        results = []
    
    if isinstance(obj, dict):
        for key, value in obj.items():
            current_path = parent_keys + [key]
            
            if isinstance(value, dict):
                # 딕셔너리인 경우 재귀 호출
                extract_nested_values(value, current_path, results)
            elif isinstance(value, list):
                # 리스트인 경우
                if len(value) > 0:
                    # 리스트의 첫 번째 요소만 처리
                    first_item = value[0]
                    if isinstance(first_item, dict):
                        # 딕셔너리가 포함된 리스트인 경우
                        extract_nested_values(first_item, current_path, results)
                    else:
                        # 단순 값의 리스트인 경우
                        results.append({
                            'level': len(current_path),
                            'key': '; '.join(current_path),
                            'value': str(first_item)
                        })
                else:
                    # 빈 리스트
                    results.append({
                        'level': len(current_path),
                        'key': '; '.join(current_path),
                        'value': None
                    })
            else:
                # 단순 값인 경우 (leaf node)
                results.append({
                    'level': len(current_path),
                    'key': '; '.join(current_path),
                    'value': str(value) if value is not None else None
                })
    elif isinstance(obj, list):
        # 최상위가 리스트인 경우
        if len(obj) > 0 and isinstance(obj[0], dict):
            extract_nested_values(obj[0], parent_keys, results)
    
    return results

def create_hierarchical_dataframe(df_products, max_rows=None):
    """제품별로 계층 구조화된 데이터프레임 생성"""
    all_rows = []
    
    # 처리할 행 수 제한
    rows_to_process = df_products if max_rows is None else df_products.head(max_rows)
    
    for idx, row in rows_to_process.iterrows():
        try:
            spec_str = row['product_specification']
            if pd.notna(spec_str):
                # JSON 파싱
                if isinstance(spec_str, str):
                    spec_json = json.loads(spec_str)
                else:
                    spec_json = spec_str
                
                # 중첩된 모든 값 추출
                nested_values = extract_nested_values(spec_json)
                
                # 각 추출된 값에 대해 행 생성
                for item in nested_values:
                    row_data = {
                        'product_id': row['product_id'],
                        'display_category_major': row['display_category_major'],
                        'display_category_middle': row['display_category_middle'],
                        'display_category_minor': row['display_category_minor'],
                        'level': item['level'],
                        'key': item['key'],
                        'value': item['value']
                    }
                    all_rows.append(row_data)
                    
        except Exception as e:
            print(f"  ⚠️ 행 {idx} 처리 실패: {e}")
            continue
    
    # 데이터프레임 생성 및 정렬
    df_result = pd.DataFrame(all_rows)
    if not df_result.empty:
        df_result = df_result.sort_values(['product_id', 'key'])
    
    return df_result

def get_max_depth(obj, current_depth=0):
    """JSON 객체의 최대 깊이 계산"""
    if not isinstance(obj, (dict, list)):
        return current_depth
    
    if isinstance(obj, dict):
        if not obj:
            return current_depth
        return max(get_max_depth(v, current_depth + 1) for v in obj.values())
    
    if isinstance(obj, list):
        if not obj:
            return current_depth
        return max(get_max_depth(item, current_depth) for item in obj)

# DB에서 데이터 로드 및 분석 실행
conn = get_db_connection()
if conn:
    try:
        # product_specification이 not null인 데이터만 조회
        query = """
        SELECT 
            product_id,
            display_category_major,
            display_category_middle,
            display_category_minor,
            product_specification
        FROM kt_merged_product_20250929
        WHERE product_specification IS NOT NULL
        LIMIT 5000
        """
        
        df_products = pd.read_sql_query(query, conn)
        print(f"📊 데이터 로드 완료: {len(df_products)}개 행")
        
        # 최대 계층 깊이 분석
        print("\n⏳ JSON 계층 깊이 분석 중...")
        max_depth = 0
        depth_distribution = {}
        
        for spec in df_products['product_specification']:
            try:
                if isinstance(spec, str):
                    spec_json = json.loads(spec)
                else:
                    spec_json = spec
                
                depth = get_max_depth(spec_json)
                max_depth = max(max_depth, depth)
                
                if depth not in depth_distribution:
                    depth_distribution[depth] = 0
                depth_distribution[depth] += 1
            except:
                continue
        
        print(f"\n📏 계층 깊이 분석 결과:")
        print(f"   - 최대 계층 깊이: {max_depth}")
        for depth, count in sorted(depth_distribution.items()):
            print(f"   - 깊이 {depth}: {count}개 제품")
        
        # 계층 구조 데이터프레임 생성
        print("\n⏳ 계층 구조 데이터프레임 생성 중...")
        df_hierarchical = create_hierarchical_dataframe(df_products, max_rows=1000)
        
        print(f"\n✅ 분석 완료!")
        print(f"   - 총 레코드 수: {len(df_hierarchical):,}")
        print(f"   - 고유 제품 수: {df_hierarchical['product_id'].nunique():,}")
        print(f"   - 고유 key 수: {df_hierarchical['key'].nunique():,}")
        
        # 레벨별 통계
        print(f"\n📊 레벨별 데이터 분포:")
        level_stats = df_hierarchical['level'].value_counts().sort_index()
        for level, count in level_stats.items():
            print(f"   - Level {level}: {count:,}개 레코드")
        
        # 샘플 데이터 표시
        print("\n📋 샘플 데이터 (처음 30개 행):")
        display(df_hierarchical.head(30))
        
        # 다양한 레벨의 예시 보여주기
        print("\n🔍 레벨별 데이터 예시:")
        for level in sorted(df_hierarchical['level'].unique()):
            level_sample = df_hierarchical[df_hierarchical['level'] == level].head(3)
            if not level_sample.empty:
                print(f"\n   Level {level} 예시:")
                for _, row in level_sample.iterrows():
                    print(f"      • key: '{row['key']}' → value: '{row['value']}'")
        
        # 특정 제품의 전체 구조 보기
        sample_product = df_hierarchical['product_id'].iloc[0]
        print(f"\n🔍 제품 {sample_product}의 전체 specification 구조:")
        sample_spec = df_hierarchical[df_hierarchical['product_id'] == sample_product][['level', 'key', 'value']]
        display(sample_spec)
        
        # 중첩된 key 분석 (';'를 포함하는 key들)
        nested_keys = df_hierarchical[df_hierarchical['key'].str.contains(';', na=False)]
        if not nested_keys.empty:
            print(f"\n🔗 중첩된 구조를 가진 key 통계:")
            print(f"   - 총 중첩 key 수: {len(nested_keys):,}")
            print(f"   - 고유 중첩 key 패턴: {nested_keys['key'].nunique():,}")
            
            # 중첩 구조 예시
            print("\n   중첩 구조 예시 (상위 10개):")
            nested_examples = nested_keys.groupby('key').size().sort_values(ascending=False).head(10)
            for key, count in nested_examples.items():
                print(f"      • {key}: {count}개")
        
        # 카테고리별 주요 key 분석
        print("\n📊 카테고리별 주요 key 통계:")
        category_key_stats = df_hierarchical.groupby(['display_category_major', 'key']).size().reset_index(name='count')
        
        major_categories = df_hierarchical['display_category_major'].value_counts().head(3).index
        for category in major_categories:
            cat_data = category_key_stats[category_key_stats['display_category_major'] == category]
            top_keys = cat_data.nlargest(5, 'count')
            
            print(f"\n   [{category}] - 상위 5개 key:")
            for _, row in top_keys.iterrows():
                level_indicator = "  " * (row['key'].count(';'))  # 들여쓰기로 레벨 표시
                print(f"      {level_indicator}• {row['key']}: {row['count']}개")
        
        # 가장 깊은 중첩 구조 찾기
        if not nested_keys.empty:
            max_nesting = nested_keys['key'].str.count(';').max() + 1
            deepest_keys = nested_keys[nested_keys['key'].str.count(';') == (max_nesting - 1)]
            
            print(f"\n🏔️ 가장 깊은 중첩 구조 (Level {max_nesting}):")
            for _, row in deepest_keys.head(5).iterrows():
                print(f"   • {row['key']}")
                print(f"     → value: {row['value']}")
                print(f"     → category: {row['display_category_major']}")
        
        # 결과를 변수에 저장
        df_hierarchical_final = df_hierarchical
        
        print("\n💾 데이터프레임이 다음 변수에 저장되었습니다:")
        print("   - df_hierarchical_final: 계층 구조 데이터프레임")
        print("     • level: 깊이 (1, 2, 3...)")
        print("     • key: 계층 경로 (';'로 구분)")
        print("     • value: 실제 값")
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        import traceback
        traceback.print_exc()
    finally:
        conn.close()
else:
    print("❌ DB 연결 실패")

In [ ]:
# 12. product_specification 깊이가 2 이상인 제품 찾기
import json
import pandas as pd

def get_json_depth(obj, current_depth=0):
    """JSON 객체의 최대 깊이를 재귀적으로 계산"""
    if not isinstance(obj, (dict, list)):
        return current_depth
    
    if isinstance(obj, dict):
        if not obj:
            return current_depth
        return max(get_json_depth(v, current_depth + 1) for v in obj.values())
    
    if isinstance(obj, list):
        if not obj:
            return current_depth
        return max(get_json_depth(item, current_depth) for item in obj)

# DB에서 데이터 로드
conn = get_db_connection()
if conn:
    try:
        # product_specification이 not null인 모든 데이터 조회
        query = """
        SELECT 
            product_id,
            display_category_major,
            display_category_middle,
            display_category_minor,
            model_name,
            product_name,
            product_specification
        FROM kt_merged_product_20250929
        WHERE product_specification IS NOT NULL
        """
        
        df_products = pd.read_sql_query(query, conn)
        print(f"📊 전체 데이터 로드 완료: {len(df_products)}개 행\n")
        
        # 각 제품의 JSON 깊이 계산
        products_with_depth = []
        
        for idx, row in df_products.iterrows():
            try:
                spec_str = row['product_specification']
                if pd.notna(spec_str):
                    # JSON 파싱
                    if isinstance(spec_str, str):
                        spec_json = json.loads(spec_str)
                    else:
                        spec_json = spec_str
                    
                    # 깊이 계산
                    depth = get_json_depth(spec_json)
                    
                    products_with_depth.append({
                        'product_id': row['product_id'],
                        'display_category_major': row['display_category_major'],
                        'display_category_middle': row['display_category_middle'],
                        'display_category_minor': row['display_category_minor'],
                        'model_name': row['model_name'],
                        'product_name': row['product_name'],
                        'depth': depth,
                        'spec_sample': str(spec_json)[:200] + '...' if len(str(spec_json)) > 200 else str(spec_json)
                    })
            except Exception as e:
                print(f"  ⚠️ 행 {idx} (product_id: {row['product_id']}) 처리 실패: {e}")
                continue
        
        # 데이터프레임 생성
        df_depth = pd.DataFrame(products_with_depth)
        
        # 깊이별 통계
        print("📊 JSON 깊이별 제품 분포:")
        depth_stats = df_depth['depth'].value_counts().sort_index()
        for depth, count in depth_stats.items():
            percentage = (count / len(df_depth)) * 100
            print(f"   - 깊이 {depth}: {count:4d}개 ({percentage:5.1f}%)")
        
        # 깊이가 2 이상인 제품 필터링
        df_deep = df_depth[df_depth['depth'] >= 2].copy()
        
        print(f"\n🔍 깊이가 2 이상인 제품: {len(df_deep)}개")
        
        if len(df_deep) > 0:
            # 카테고리별 분포
            print("\n📦 카테고리별 분포 (깊이 >= 2):")
            category_dist = df_deep['display_category_major'].value_counts()
            for category, count in category_dist.head(10).items():
                print(f"   • {category}: {count}개")
            
            # 깊이별 상세 분석
            print("\n📋 깊이별 제품 목록:")
            
            for depth in sorted(df_deep['depth'].unique(), reverse=True):
                depth_products = df_deep[df_deep['depth'] == depth]
                print(f"\n━━━ 깊이 {depth} ({len(depth_products)}개 제품) ━━━")
                
                # 최대 10개만 표시
                for idx, (_, row) in enumerate(depth_products.head(10).iterrows(), 1):
                    print(f"\n{idx}. Product ID: {row['product_id']}")
                    print(f"   - 제품명: {row['product_name']}")
                    print(f"   - 모델명: {row['model_name']}")
                    print(f"   - 카테고리: {row['display_category_major']} > {row['display_category_middle']} > {row['display_category_minor']}")
                    print(f"   - JSON 샘플: {row['spec_sample']}")
                
                if len(depth_products) > 10:
                    print(f"\n   ... 외 {len(depth_products) - 10}개 더 있음")
            
            # 깊이가 가장 깊은 제품 상세 분석
            max_depth = df_deep['depth'].max()
            deepest_products = df_deep[df_deep['depth'] == max_depth]
            
            print(f"\n🏔️ 가장 깊은 구조를 가진 제품 (깊이 {max_depth}):")
            for idx, (_, row) in enumerate(deepest_products.iterrows(), 1):
                print(f"\n{idx}. {row['product_id']} - {row['product_name']}")
                
                # 실제 JSON 구조 파싱하여 보여주기
                spec_data = df_products[df_products['product_id'] == row['product_id']]['product_specification'].iloc[0]
                if isinstance(spec_data, str):
                    spec_json = json.loads(spec_data)
                else:
                    spec_json = spec_data
                
                # JSON 구조를 들여쓰기로 보기 좋게 출력
                print("   JSON 구조:")
                print(json.dumps(spec_json, indent=2, ensure_ascii=False)[:1000])
                if len(json.dumps(spec_json)) > 1000:
                    print("   ... (이하 생략)")
            
            # 결과 데이터프레임 표시
            print("\n📊 깊이 2 이상 제품 데이터프레임 (상위 20개):")
            display(df_deep[['product_id', 'product_name', 'display_category_major', 'display_category_middle', 'depth']].head(20))
            
            # CSV로 저장할 수 있도록 준비
            df_deep_export = df_deep[['product_id', 'product_name', 'model_name', 
                                      'display_category_major', 'display_category_middle', 
                                      'display_category_minor', 'depth']]
            
            print(f"\n💾 변수에 저장됨:")
            print(f"   - df_deep_products: 깊이 2 이상인 제품 전체 데이터")
            print(f"   - df_deep_export: 내보내기용 데이터 (spec_sample 제외)")
            
            # 변수에 저장
            df_deep_products = df_deep
            df_deep_export = df_deep_export
            
        else:
            print("\n✅ 모든 제품의 JSON 깊이가 1입니다.")
            df_deep_products = pd.DataFrame()
            df_deep_export = pd.DataFrame()
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        import traceback
        traceback.print_exc()
    finally:
        conn.close()
else:
    print("❌ DB 연결 실패")

In [ ]:
# 13. 카테고리별 상위 10개 key의 value 분포 및 outlier 분석
import json
import pandas as pd
import numpy as np
from collections import Counter

def detect_outliers_in_values(values_list):
    """값 리스트에서 outlier를 감지"""
    if len(values_list) < 3:
        return []
    
    # 숫자형 값 시도
    numeric_values = []
    for v in values_list:
        try:
            # 단위 제거하고 숫자만 추출 시도
            import re
            num_match = re.findall(r'[-+]?\d*\.?\d+', str(v))
            if num_match:
                numeric_values.append(float(num_match[0]))
        except:
            pass
    
    # 숫자형 outlier 감지 (IQR 방법)
    if len(numeric_values) > 3:
        q1 = np.percentile(numeric_values, 25)
        q3 = np.percentile(numeric_values, 75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        
        outliers = []
        for i, val in enumerate(numeric_values):
            if val < lower_bound or val > upper_bound:
                outliers.append((values_list[i], val, 'numeric'))
        return outliers
    
    # 문자형 값의 빈도 기반 outlier (매우 드문 값)
    value_counts = Counter(values_list)
    total = len(values_list)
    outliers = []
    
    for value, count in value_counts.items():
        if count == 1 and total > 10:  # 전체 10개 이상 중 1번만 나타난 값
            outliers.append((value, count/total, 'frequency'))
    
    return outliers

# DB에서 데이터 로드
conn = get_db_connection()
if conn:
    try:
        # product_specification이 있는 모든 데이터 조회
        query = """
        SELECT 
            product_id,
            display_category_major,
            display_category_middle,
            display_category_minor,
            product_name,
            product_specification
        FROM kt_merged_product_20250929
        WHERE product_specification IS NOT NULL
        """
        
        df_products = pd.read_sql_query(query, conn)
        print(f"📊 전체 데이터 로드 완료: {len(df_products)}개 행\n")
        
        # 카테고리별로 데이터 정리
        category_key_values = {}
        
        for idx, row in df_products.iterrows():
            try:
                category = row['display_category_major']
                spec_str = row['product_specification']
                
                if pd.notna(spec_str):
                    # JSON 파싱
                    if isinstance(spec_str, str):
                        spec_json = json.loads(spec_str)
                    else:
                        spec_json = spec_str
                    
                    if category not in category_key_values:
                        category_key_values[category] = {}
                    
                    # 각 key-value 저장
                    for key, value in spec_json.items():
                        if key not in category_key_values[category]:
                            category_key_values[category][key] = []
                        
                        # value 처리
                        if isinstance(value, list) and len(value) > 0:
                            value_str = str(value[0])
                        elif isinstance(value, (dict, list)):
                            value_str = json.dumps(value, ensure_ascii=False)
                        else:
                            value_str = str(value) if value is not None else None
                        
                        if value_str:
                            category_key_values[category][key].append({
                                'product_id': row['product_id'],
                                'product_name': row['product_name'],
                                'value': value_str
                            })
                            
            except Exception as e:
                continue
        
        # 각 카테고리별 분석
        print("=" * 80)
        print("📊 카테고리별 상위 10개 Key의 Value 분포 및 Outlier 분석")
        print("=" * 80)
        
        all_outliers = []
        
        for category in sorted(category_key_values.keys()):
            print(f"\n\n{'='*60}")
            print(f"📦 카테고리: {category}")
            print(f"{'='*60}")
            
            # 해당 카테고리의 key를 빈도순으로 정렬
            key_frequencies = [(key, len(values)) for key, values in category_key_values[category].items()]
            top_keys = sorted(key_frequencies, key=lambda x: x[1], reverse=True)[:10]
            
            print(f"총 {len(key_frequencies)}개 key 중 상위 10개 분석\n")
            
            category_outliers = []
            
            for i, (key, freq) in enumerate(top_keys, 1):
                values_data = category_key_values[category][key]
                values_list = [v['value'] for v in values_data]
                unique_values = list(set(values_list))
                
                print(f"\n{i}. Key: '{key}'")
                print(f"   - 총 제품 수: {len(values_list)}")
                print(f"   - 고유 값 개수: {len(unique_values)}")
                
                # value 분포 분석
                value_counter = Counter(values_list)
                
                # 상위 5개 빈도 값
                top_values = value_counter.most_common(5)
                print(f"   - 최빈 값 Top 5:")
                for value, count in top_values:
                    percentage = (count / len(values_list)) * 100
                    display_value = value[:50] + '...' if len(value) > 50 else value
                    print(f"      • '{display_value}': {count}개 ({percentage:.1f}%)")
                
                # Outlier 감지
                outliers = detect_outliers_in_values(values_list)
                
                if outliers:
                    print(f"   \n   🔴 Outlier 발견 ({len(outliers)}개):")
                    
                    # outlier 타입별로 정리
                    numeric_outliers = [o for o in outliers if o[2] == 'numeric']
                    frequency_outliers = [o for o in outliers if o[2] == 'frequency']
                    
                    if numeric_outliers:
                        print(f"      [수치형 Outlier]")
                        for out_value, numeric_val, _ in numeric_outliers[:5]:
                            # 해당 값을 가진 제품 찾기
                            products = [v for v in values_data if v['value'] == out_value]
                            if products:
                                product = products[0]
                                print(f"         - 값: '{out_value}' (수치: {numeric_val:.2f})")
                                print(f"           제품: {product['product_id']} - {product['product_name'][:50]}")
                            
                            # 전체 outlier 리스트에 추가
                            category_outliers.append({
                                'category': category,
                                'key': key,
                                'value': out_value,
                                'outlier_type': 'numeric',
                                'product_id': product['product_id'] if products else None,
                                'product_name': product['product_name'] if products else None
                            })
                    
                    if frequency_outliers and len(unique_values) > 10:
                        print(f"      [빈도 기반 Outlier (희귀 값)]")
                        for out_value, freq_ratio, _ in frequency_outliers[:5]:
                            # 해당 값을 가진 제품 찾기
                            products = [v for v in values_data if v['value'] == out_value]
                            if products:
                                product = products[0]
                                display_value = out_value[:50] + '...' if len(out_value) > 50 else out_value
                                print(f"         - 값: '{display_value}'")
                                print(f"           제품: {product['product_id']} - {product['product_name'][:50]}")
                            
                            # 전체 outlier 리스트에 추가
                            category_outliers.append({
                                'category': category,
                                'key': key,
                                'value': out_value,
                                'outlier_type': 'frequency',
                                'product_id': product['product_id'] if products else None,
                                'product_name': product['product_name'] if products else None
                            })
                
                # 값의 패턴 분석
                if len(unique_values) <= 10:
                    print(f"   - 모든 고유 값:")
                    for value in sorted(unique_values)[:10]:
                        display_value = value[:50] + '...' if len(value) > 50 else value
                        count = value_counter[value]
                        print(f"      • '{display_value}': {count}개")
            
            all_outliers.extend(category_outliers)
            
            # 카테고리별 outlier 요약
            if category_outliers:
                print(f"\n   📊 {category} 카테고리 Outlier 요약:")
                print(f"      - 총 {len(category_outliers)}개 outlier 발견")
                outlier_keys = Counter([o['key'] for o in category_outliers])
                print(f"      - Outlier가 있는 key: {', '.join(outlier_keys.keys())}")
        
        # 전체 Outlier 데이터프레임 생성
        if all_outliers:
            df_outliers = pd.DataFrame(all_outliers)
            
            print("\n" + "=" * 80)
            print("📊 전체 Outlier 요약")
            print("=" * 80)
            
            print(f"\n총 {len(df_outliers)}개 outlier 발견")
            
            print("\n카테고리별 outlier 수:")
            category_counts = df_outliers['category'].value_counts()
            for cat, count in category_counts.items():
                print(f"   • {cat}: {count}개")
            
            print("\nOutlier 타입별 분포:")
            type_counts = df_outliers['outlier_type'].value_counts()
            for type_name, count in type_counts.items():
                print(f"   • {type_name}: {count}개")
            
            print("\n상위 10개 Outlier 상세:")
            display(df_outliers[['category', 'key', 'value', 'outlier_type', 'product_id', 'product_name']].head(10))
            
            # 특정 key에서 자주 outlier가 발생하는지 확인
            key_outlier_counts = df_outliers.groupby(['category', 'key']).size().sort_values(ascending=False).head(10)
            
            print("\nOutlier가 많이 발생한 Key Top 10:")
            for (cat, key), count in key_outlier_counts.items():
                print(f"   • [{cat}] {key}: {count}개 outlier")
            
            print("\n💾 변수에 저장됨:")
            print("   - df_outliers: 전체 outlier 데이터프레임")
            print("   - category_key_values: 카테고리별 key-value 전체 데이터")
        else:
            print("\n✅ 명확한 outlier가 발견되지 않았습니다.")
            df_outliers = pd.DataFrame()
            
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        import traceback
        traceback.print_exc()
    finally:
        conn.close()
else:
    print("❌ DB 연결 실패")

In [ ]:
# 14. display_category_major, display_category_middle, key별 분포 및 outlier 분석
import json
import pandas as pd
import numpy as np
from collections import Counter

def detect_distribution_outliers(groups_data):
    """그룹별 count 분포에서 outlier 감지"""
    counts = [g['count'] for g in groups_data]
    
    if len(counts) < 4:
        return []
    
    # IQR 방법으로 outlier 감지
    q1 = np.percentile(counts, 25)
    q3 = np.percentile(counts, 75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    outliers = []
    for group in groups_data:
        if group['count'] < lower_bound or group['count'] > upper_bound:
            group['outlier_type'] = 'count_outlier'
            group['lower_bound'] = lower_bound
            group['upper_bound'] = upper_bound
            outliers.append(group)
    
    return outliers

# DB에서 데이터 로드
conn = get_db_connection()
if conn:
    try:
        # product_specification이 있는 모든 데이터 조회
        query = """
        SELECT 
            product_id,
            display_category_major,
            display_category_middle,
            display_category_minor,
            product_name,
            product_specification
        FROM kt_merged_product_20250929
        WHERE product_specification IS NOT NULL
        """
        
        df_products = pd.read_sql_query(query, conn)
        print(f"📊 전체 데이터 로드 완료: {len(df_products)}개 행\n")
        
        # 카테고리(major, middle)와 key별로 데이터 정리
        category_key_data = []
        
        for idx, row in df_products.iterrows():
            try:
                spec_str = row['product_specification']
                
                if pd.notna(spec_str):
                    # JSON 파싱
                    if isinstance(spec_str, str):
                        spec_json = json.loads(spec_str)
                    else:
                        spec_json = spec_str
                    
                    # 각 key에 대해 데이터 수집
                    for key, value in spec_json.items():
                        # value 처리
                        if isinstance(value, list) and len(value) > 0:
                            value_str = str(value[0])
                        elif isinstance(value, (dict, list)):
                            value_str = json.dumps(value, ensure_ascii=False)
                        else:
                            value_str = str(value) if value is not None else None
                        
                        if value_str:
                            category_key_data.append({
                                'display_category_major': row['display_category_major'],
                                'display_category_middle': row['display_category_middle'],
                                'display_category_minor': row['display_category_minor'],
                                'key': key,
                                'value': value_str,
                                'product_id': row['product_id'],
                                'product_name': row['product_name']
                            })
                            
            except Exception as e:
                continue
        
        # 데이터프레임 생성
        df_category_key = pd.DataFrame(category_key_data)
        
        print("=" * 80)
        print("📊 Category Major, Middle, Key별 분포 분석")
        print("=" * 80)
        
        # Group by major, middle, key
        grouped = df_category_key.groupby(['display_category_major', 'display_category_middle', 'key']).agg({
            'product_id': 'count',
            'value': lambda x: len(set(x))  # 고유 value 개수
        }).rename(columns={'product_id': 'count', 'value': 'unique_values'}).reset_index()
        
        print(f"\n총 {len(grouped)}개의 (major, middle, key) 조합\n")
        
        # 기본 통계
        print("📈 Count 분포 통계:")
        print(f"   - 평균: {grouped['count'].mean():.2f}")
        print(f"   - 중앙값: {grouped['count'].median():.2f}")
        print(f"   - 최소값: {grouped['count'].min()}")
        print(f"   - 최대값: {grouped['count'].max()}")
        print(f"   - 표준편차: {grouped['count'].std():.2f}")
        
        # Count 기준 상위/하위 그룹
        print("\n📊 Count 기준 Top 10 조합:")
        top_combinations = grouped.nlargest(10, 'count')
        for idx, row in top_combinations.iterrows():
            print(f"   {row['count']:3d}개: [{row['display_category_major']}] > [{row['display_category_middle']}] > '{row['key']}'")
        
        print("\n📊 Count 기준 Bottom 10 조합 (가장 적은):")
        bottom_combinations = grouped.nsmallest(10, 'count')
        for idx, row in bottom_combinations.iterrows():
            print(f"   {row['count']:3d}개: [{row['display_category_major']}] > [{row['display_category_middle']}] > '{row['key']}'")
        
        # 각 major 카테고리별 outlier 분석
        print("\n" + "=" * 80)
        print("🔍 Major 카테고리별 Outlier 분석")
        print("=" * 80)
        
        all_outliers = []
        
        for major_cat in grouped['display_category_major'].unique():
            major_data = grouped[grouped['display_category_major'] == major_cat]
            
            if len(major_data) < 4:
                continue
            
            print(f"\n📦 {major_cat}")
            print(f"   - 총 {len(major_data)}개 (middle, key) 조합")
            
            # Count 기반 outlier 찾기
            groups_list = major_data.to_dict('records')
            outliers = detect_distribution_outliers(groups_list)
            
            if outliers:
                print(f"   - 🔴 {len(outliers)}개 outlier 발견")
                
                # 높은 outlier (비정상적으로 많은)
                high_outliers = [o for o in outliers if o['count'] > o['upper_bound']]
                if high_outliers:
                    print(f"\n   [비정상적으로 많은 count]")
                    for out in sorted(high_outliers, key=lambda x: x['count'], reverse=True)[:5]:
                        print(f"      • {out['count']}개: [{out['display_category_middle']}] > '{out['key']}'")
                        print(f"        (정상 범위: {out['lower_bound']:.1f} ~ {out['upper_bound']:.1f})")
                
                # 낮은 outlier (비정상적으로 적은)
                low_outliers = [o for o in outliers if o['count'] < o['lower_bound']]
                if low_outliers:
                    print(f"\n   [비정상적으로 적은 count]")
                    for out in sorted(low_outliers, key=lambda x: x['count'])[:5]:
                        print(f"      • {out['count']}개: [{out['display_category_middle']}] > '{out['key']}'")
                
                all_outliers.extend(outliers)
        
        # Middle 카테고리별 분석
        print("\n" + "=" * 80)
        print("🔍 Middle 카테고리별 Key 분포 분석")
        print("=" * 80)
        
        middle_key_stats = grouped.groupby('display_category_middle').agg({
            'key': 'count',
            'count': ['sum', 'mean', 'std']
        }).round(2)
        
        middle_key_stats.columns = ['unique_keys', 'total_products', 'avg_products_per_key', 'std_products']
        middle_key_stats = middle_key_stats.sort_values('total_products', ascending=False)
        
        print("\nMiddle 카테고리별 통계 (상위 15개):")
        display(middle_key_stats.head(15))
        
        # Key별 카테고리 분포 분석
        print("\n" + "=" * 80)
        print("🔑 Key별 카테고리 분포 분석")
        print("=" * 80)
        
        key_category_stats = grouped.groupby('key').agg({
            'display_category_major': 'nunique',
            'display_category_middle': 'nunique',
            'count': ['sum', 'mean', 'std']
        }).round(2)
        
        key_category_stats.columns = ['major_categories', 'middle_categories', 'total_products', 'avg_products', 'std_products']
        key_category_stats = key_category_stats.sort_values('total_products', ascending=False)
        
        print("\nKey별 카테고리 분포 (상위 20개):")
        display(key_category_stats.head(20))
        
        # 특이 패턴 찾기
        print("\n" + "=" * 80)
        print("🎯 특이 패턴 분석")
        print("=" * 80)
        
        # 1. 한 middle 카테고리에만 있는 unique key
        unique_keys_per_middle = grouped.groupby('key')['display_category_middle'].nunique()
        single_middle_keys = unique_keys_per_middle[unique_keys_per_middle == 1].index
        
        if len(single_middle_keys) > 0:
            print(f"\n1. 단일 Middle 카테고리 전용 Key ({len(single_middle_keys)}개 중 10개):")
            for key in list(single_middle_keys)[:10]:
                middle_cat = grouped[grouped['key'] == key]['display_category_middle'].iloc[0]
                count = grouped[grouped['key'] == key]['count'].iloc[0]
                print(f"   • '{key}' → [{middle_cat}] ({count}개 제품)")
        
        # 2. 여러 카테고리에 걸쳐있는 key
        multi_category_keys = key_category_stats[key_category_stats['middle_categories'] >= 5].head(10)
        
        if len(multi_category_keys) > 0:
            print(f"\n2. 5개 이상 Middle 카테고리에 걸친 범용 Key:")
            for key, row in multi_category_keys.iterrows():
                print(f"   • '{key}': {int(row['middle_categories'])}개 middle 카테고리, {int(row['total_products'])}개 제품")
        
        # 3. Value 다양성이 높은 조합
        high_diversity = grouped.nlargest(10, 'unique_values')
        
        print(f"\n3. Value 다양성이 높은 조합 (고유값이 많은):")
        for idx, row in high_diversity.iterrows():
            print(f"   • {row['unique_values']}개 고유값: [{row['display_category_major']}] > [{row['display_category_middle']}] > '{row['key']}'")
        
        # Outlier 데이터프레임 생성
        if all_outliers:
            df_group_outliers = pd.DataFrame(all_outliers)
            print(f"\n\n💾 결과 저장:")
            print(f"   - df_grouped: 전체 그룹별 통계 데이터프레임")
            print(f"   - df_group_outliers: 그룹 outlier 데이터프레임")
            print(f"   - middle_key_stats: Middle 카테고리별 통계")
            print(f"   - key_category_stats: Key별 카테고리 분포 통계")
        else:
            df_group_outliers = pd.DataFrame()
            print(f"\n\n💾 결과 저장:")
            print(f"   - df_grouped: 전체 그룹별 통계 데이터프레임")
            print(f"   - middle_key_stats: Middle 카테고리별 통계")
            print(f"   - key_category_stats: Key별 카테고리 분포 통계")
        
        # 변수 저장
        df_grouped = grouped
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        import traceback
        traceback.print_exc()
    finally:
        conn.close()
else:
    print("❌ DB 연결 실패")

In [ ]:
# 15. 모든 카테고리의 unique values를 분석하여 파일로 출력 (비율 포함)
import json
import pandas as pd
import os

def analyze_all_categories_to_file():
    """모든 middle 카테고리의 각 key별 unique values를 분석하여 파일로 저장 (비율 포함)"""
    
    conn = get_db_connection()
    if not conn:
        return None
    
    try:
        # 모든 데이터 조회
        query = """
        SELECT 
            display_category_major,
            display_category_middle,
            display_category_minor,
            product_id,
            product_specification
        FROM kt_merged_product_20250929
        WHERE product_specification IS NOT NULL
        """
        
        df_products = pd.read_sql_query(query, conn)
        print(f"📊 전체 데이터 로드 완료: {len(df_products)}개 제품")
        
        # 카테고리별로 제품 수 계산 (비율 계산용)
        category_product_counts = {}
        
        # 카테고리별로 unique values 수집
        category_key_values = {}
        category_key_products = {}  # 각 key가 나타난 제품 ID 추적
        
        for idx, row in df_products.iterrows():
            try:
                major = row['display_category_major']
                middle = row['display_category_middle']
                product_id = row['product_id']
                spec_str = row['product_specification']
                
                if pd.notna(spec_str):
                    # JSON 파싱
                    if isinstance(spec_str, str):
                        spec_json = json.loads(spec_str)
                    else:
                        spec_json = spec_str
                    
                    # 카테고리별 제품 수 카운트
                    if major not in category_product_counts:
                        category_product_counts[major] = {'total': set(), 'middle': {}}
                    if middle not in category_product_counts[major]['middle']:
                        category_product_counts[major]['middle'][middle] = set()
                    
                    category_product_counts[major]['total'].add(product_id)
                    category_product_counts[major]['middle'][middle].add(product_id)
                    
                    # 카테고리 조합 키 생성
                    category_key = (major, middle)
                    
                    if category_key not in category_key_values:
                        category_key_values[category_key] = {}
                        category_key_products[category_key] = {}
                    
                    # 각 key-value 수집
                    for key, value in spec_json.items():
                        if key not in category_key_values[category_key]:
                            category_key_values[category_key][key] = set()
                            category_key_products[category_key][key] = set()
                        
                        # value 처리
                        if isinstance(value, list) and len(value) > 0:
                            value_str = str(value[0])
                        elif isinstance(value, (dict, list)):
                            value_str = json.dumps(value, ensure_ascii=False)
                        else:
                            value_str = str(value) if value is not None else None
                        
                        if value_str:
                            category_key_values[category_key][key].add(value_str)
                            category_key_products[category_key][key].add(product_id)
                            
            except Exception as e:
                continue
        
        print(f"✅ 데이터 수집 완료: {len(category_key_values)}개 카테고리 조합")
        
        # 카테고리별 총 제품 수 계산
        category_totals = {}
        for major, data in category_product_counts.items():
            category_totals[major] = {
                'total': len(data['total']),
                'middle': {middle: len(products) for middle, products in data['middle'].items()}
            }
        
        # 결과를 정리하여 리스트로 변환
        result_data = []
        
        for (major, middle), keys_dict in category_key_values.items():
            major_total = category_totals[major]['total']
            middle_total = category_totals[major]['middle'][middle]
            
            for key, values in keys_dict.items():
                sorted_values = sorted(list(values))
                
                # 이 key를 가진 제품 수
                products_with_key = len(category_key_products[(major, middle)][key])
                
                # 비율 계산
                ratio_in_middle = (products_with_key / middle_total * 100) if middle_total > 0 else 0
                ratio_in_major = (products_with_key / major_total * 100) if major_total > 0 else 0
                
                # 탭으로 구분된 값 리스트 생성
                # 값들 사이는 세미콜론(;)으로 구분
                values_str = '; '.join(sorted_values)
                
                result_data.append({
                    'display_category_major': major,
                    'display_category_middle': middle,
                    'key': key,
                    'product_count': products_with_key,
                    'middle_total': middle_total,
                    'ratio_in_middle': round(ratio_in_middle, 2),
                    'major_total': major_total,
                    'ratio_in_major': round(ratio_in_major, 2),
                    'unique_values_count': len(sorted_values),
                    'unique_values': values_str
                })
        
        # 데이터프레임 생성 및 정렬
        df_result = pd.DataFrame(result_data)
        df_result = df_result.sort_values(['display_category_major', 'display_category_middle', 'key'])
        
        # 통계 출력
        print(f"\n📊 분석 통계:")
        print(f"   - 총 레코드 수: {len(df_result):,}")
        print(f"   - Major 카테고리 수: {df_result['display_category_major'].nunique()}")
        print(f"   - Middle 카테고리 수: {df_result['display_category_middle'].nunique()}")
        print(f"   - 고유 Key 수: {df_result['key'].nunique()}")
        
        # 카테고리별 통계
        print(f"\n📋 카테고리별 통계:")
        category_stats = df_result.groupby(['display_category_major', 'display_category_middle']).agg({
            'key': 'count',
            'middle_total': 'first'
        }).rename(columns={'key': 'key_count', 'middle_total': 'product_count'}).reset_index()
        category_stats = category_stats.sort_values('product_count', ascending=False)
        
        print("\n상위 10개 카테고리:")
        for idx, row in category_stats.head(10).iterrows():
            print(f"   • [{row['display_category_major']}] {row['display_category_middle']}: "
                  f"{row['product_count']}개 제품, {row['key_count']}개 key")
        
        # 파일로 저장 (탭 구분)
        output_file = 'category_middle_spec_analysis.txt'
        
        # 저장할 컬럼 순서 정의
        columns_order = [
            'display_category_major', 
            'display_category_middle', 
            'key', 
            'product_count',
            'middle_total',
            'ratio_in_middle',
            'major_total', 
            'ratio_in_major',
            'unique_values_count',
            'unique_values'
        ]
        
        # 탭으로 구분된 텍스트 파일로 저장
        df_result[columns_order].to_csv(output_file, sep='\t', index=False, encoding='utf-8')
        
        print(f"\n💾 파일 저장 완료: {output_file}")
        print(f"   - 파일 크기: {os.path.getsize(output_file) / 1024:.2f} KB")
        print(f"   - 총 {len(df_result):,}개 행")
        print(f"   - 엑셀에서 열 때: 텍스트 가져오기 → 탭으로 구분")
        
        # 샘플 데이터 표시 (비율 포함)
        print(f"\n📋 데이터 샘플 (처음 10개 행):")
        display_cols = ['display_category_major', 'display_category_middle', 'key', 
                       'product_count', 'ratio_in_middle', 'ratio_in_major', 'unique_values_count']
        display(df_result[display_cols].head(10))
        
        # 비율이 높은 key들 분석
        print(f"\n📈 Middle 카테고리 내 비율이 높은 Key Top 10 (90% 이상):")
        high_ratio = df_result[df_result['ratio_in_middle'] >= 90].sort_values('ratio_in_middle', ascending=False)
        for idx, row in high_ratio.head(10).iterrows():
            print(f"   • {row['ratio_in_middle']:.1f}%: [{row['display_category_major']}] "
                  f"{row['display_category_middle']} > '{row['key']}' ({row['product_count']}/{row['middle_total']})")
        
        # 비율이 낮은 key들 분석
        print(f"\n📉 Middle 카테고리 내 비율이 낮은 Key (10% 미만, 최소 5개 제품):")
        low_ratio = df_result[(df_result['ratio_in_middle'] < 10) & (df_result['product_count'] >= 5)]
        low_ratio = low_ratio.sort_values('ratio_in_middle')
        for idx, row in low_ratio.head(10).iterrows():
            print(f"   • {row['ratio_in_middle']:.1f}%: [{row['display_category_major']}] "
                  f"{row['display_category_middle']} > '{row['key']}' ({row['product_count']}/{row['middle_total']})")
        
        # 특정 카테고리 상세 보기
        print(f"\n🔍 특정 카테고리 예시 (갤럭시 스마트폰):")
        galaxy_data = df_result[df_result['display_category_middle'] == '갤럭시 스마트폰']
        if not galaxy_data.empty:
            galaxy_data = galaxy_data.sort_values('ratio_in_middle', ascending=False)
            for idx, row in galaxy_data.head(5).iterrows():
                print(f"\n   Key: '{row['key']}'")
                print(f"   제품 수: {row['product_count']}/{row['middle_total']} ({row['ratio_in_middle']:.1f}%)")
                print(f"   Major 내 비율: {row['ratio_in_major']:.1f}%")
                print(f"   고유값 개수: {row['unique_values_count']}개")
                # 값이 너무 길면 일부만 표시
                values_preview = row['unique_values']
                if len(values_preview) > 150:
                    values_preview = values_preview[:150] + '...'
                print(f"   Values: {values_preview}")
        
        # 추가로 Excel 형식으로도 저장 (선택사항)
        excel_file = 'category_middle_spec_analysis.xlsx'
        try:
            # unique_values가 너무 길면 Excel 셀 제한(32,767자)에 걸릴 수 있으므로 제한
            df_excel = df_result.copy()
            df_excel['unique_values'] = df_excel['unique_values'].apply(
                lambda x: x[:32000] + '...(truncated)' if len(x) > 32000 else x
            )
            df_excel[columns_order].to_excel(excel_file, index=False, engine='openpyxl')
            print(f"\n💾 Excel 파일도 저장됨: {excel_file}")
        except Exception as e:
            print(f"\n⚠️ Excel 저장 실패 (파일이 너무 큼): {e}")
            print(f"   → 텍스트 파일({output_file})을 사용하세요")
        
        print(f"\n✅ 분석 완료!")
        print(f"   - df_all_categories_analysis: 전체 분석 결과 데이터프레임")
        
        return df_result
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        import traceback
        traceback.print_exc()
        return None
    finally:
        conn.close()

# 모든 카테고리 분석 실행
df_all_categories_analysis = analyze_all_categories_to_file()

# 파일이 제대로 생성되었는지 확인
if os.path.exists('category_middle_spec_analysis.txt'):
    print(f"\n✅ 파일 확인: category_middle_spec_analysis.txt")
    print(f"   파일 경로: {os.path.abspath('category_middle_spec_analysis.txt')}")
    
    # 파일 처음 몇 줄 미리보기
    with open('category_middle_spec_analysis.txt', 'r', encoding='utf-8') as f:
        lines = f.readlines()[:5]
        print(f"\n📄 파일 미리보기 (처음 5줄):")
        for i, line in enumerate(lines, 1):
            # 탭으로 구분된 필드 확인
            fields = line.strip().split('\t')
            if i == 1:  # 헤더
                print(f"   헤더: {len(fields)}개 컬럼")
                print(f"   컬럼: {fields[:5]}... (처음 5개)")
            else:
                print(f"   {i}: {fields[2] if len(fields) > 2 else ''} - "
                      f"{fields[5] if len(fields) > 5 else ''}% in middle, "
                      f"{fields[7] if len(fields) > 7 else ''}% in major")

In [ ]:
# 16. display_category_minor까지 포함한 상세 분석
import json
import pandas as pd
import os

def analyze_all_categories_with_minor():
    """major, middle, minor 카테고리의 각 key별 unique values를 분석하여 파일로 저장 (비율 포함)"""
    
    conn = get_db_connection()
    if not conn:
        return None
    
    try:
        # 모든 데이터 조회
        query = """
        SELECT 
            display_category_major,
            display_category_middle,
            display_category_minor,
            product_id,
            product_specification
        FROM kt_merged_product_20250929
        WHERE product_specification IS NOT NULL
        """
        
        df_products = pd.read_sql_query(query, conn)
        print(f"📊 전체 데이터 로드 완료: {len(df_products)}개 제품")
        
        # 카테고리별로 제품 수 계산 (비율 계산용)
        category_product_counts = {}
        
        # 카테고리별로 unique values 수집
        category_key_values = {}
        category_key_products = {}  # 각 key가 나타난 제품 ID 추적
        
        for idx, row in df_products.iterrows():
            try:
                major = row['display_category_major']
                middle = row['display_category_middle']
                minor = row['display_category_minor']
                product_id = row['product_id']
                spec_str = row['product_specification']
                
                if pd.notna(spec_str):
                    # JSON 파싱
                    if isinstance(spec_str, str):
                        spec_json = json.loads(spec_str)
                    else:
                        spec_json = spec_str
                    
                    # 카테고리별 제품 수 카운트
                    if major not in category_product_counts:
                        category_product_counts[major] = {'total': set(), 'middle': {}}
                    if middle not in category_product_counts[major]['middle']:
                        category_product_counts[major]['middle'][middle] = {'total': set(), 'minor': {}}
                    if minor not in category_product_counts[major]['middle'][middle]['minor']:
                        category_product_counts[major]['middle'][middle]['minor'][minor] = set()
                    
                    category_product_counts[major]['total'].add(product_id)
                    category_product_counts[major]['middle'][middle]['total'].add(product_id)
                    category_product_counts[major]['middle'][middle]['minor'][minor].add(product_id)
                    
                    # 카테고리 조합 키 생성 (minor까지 포함)
                    category_key = (major, middle, minor)
                    
                    if category_key not in category_key_values:
                        category_key_values[category_key] = {}
                        category_key_products[category_key] = {}
                    
                    # 각 key-value 수집
                    for key, value in spec_json.items():
                        if key not in category_key_values[category_key]:
                            category_key_values[category_key][key] = set()
                            category_key_products[category_key][key] = set()
                        
                        # value 처리
                        if isinstance(value, list) and len(value) > 0:
                            value_str = str(value[0])
                        elif isinstance(value, (dict, list)):
                            value_str = json.dumps(value, ensure_ascii=False)
                        else:
                            value_str = str(value) if value is not None else None
                        
                        if value_str:
                            category_key_values[category_key][key].add(value_str)
                            category_key_products[category_key][key].add(product_id)
                            
            except Exception as e:
                continue
        
        print(f"✅ 데이터 수집 완료: {len(category_key_values)}개 카테고리 조합 (minor 포함)")
        
        # 카테고리별 총 제품 수 계산
        category_totals = {}
        for major, major_data in category_product_counts.items():
            category_totals[major] = {
                'total': len(major_data['total']),
                'middle': {}
            }
            for middle, middle_data in major_data['middle'].items():
                category_totals[major]['middle'][middle] = {
                    'total': len(middle_data['total']),
                    'minor': {minor: len(products) for minor, products in middle_data['minor'].items()}
                }
        
        # 결과를 정리하여 리스트로 변환
        result_data = []
        
        for (major, middle, minor), keys_dict in category_key_values.items():
            major_total = category_totals[major]['total']
            middle_total = category_totals[major]['middle'][middle]['total']
            minor_total = category_totals[major]['middle'][middle]['minor'].get(minor, 0)
            
            for key, values in keys_dict.items():
                sorted_values = sorted(list(values))
                
                # 이 key를 가진 제품 수
                products_with_key = len(category_key_products[(major, middle, minor)][key])
                
                # 비율 계산
                ratio_in_minor = (products_with_key / minor_total * 100) if minor_total > 0 else 0
                ratio_in_middle = (products_with_key / middle_total * 100) if middle_total > 0 else 0
                ratio_in_major = (products_with_key / major_total * 100) if major_total > 0 else 0
                
                # 탭으로 구분된 값 리스트 생성
                # 값들 사이는 세미콜론(;)으로 구분
                values_str = '; '.join(sorted_values)
                
                result_data.append({
                    'display_category_major': major,
                    'display_category_middle': middle,
                    'display_category_minor': minor,
                    'key': key,
                    'product_count': products_with_key,
                    'minor_total': minor_total,
                    'ratio_in_minor': round(ratio_in_minor, 2),
                    'middle_total': middle_total,
                    'ratio_in_middle': round(ratio_in_middle, 2),
                    'major_total': major_total,
                    'ratio_in_major': round(ratio_in_major, 2),
                    'unique_values_count': len(sorted_values),
                    'unique_values': values_str
                })
        
        # 데이터프레임 생성 및 정렬
        df_result = pd.DataFrame(result_data)
        df_result = df_result.sort_values(['display_category_major', 'display_category_middle', 'display_category_minor', 'key'])
        
        # 통계 출력
        print(f"\n📊 분석 통계 (Minor 포함):")
        print(f"   - 총 레코드 수: {len(df_result):,}")
        print(f"   - Major 카테고리 수: {df_result['display_category_major'].nunique()}")
        print(f"   - Middle 카테고리 수: {df_result['display_category_middle'].nunique()}")
        print(f"   - Minor 카테고리 수: {df_result['display_category_minor'].nunique()}")
        print(f"   - 고유 Key 수: {df_result['key'].nunique()}")
        
        # Minor 카테고리별 통계
        print(f"\n📋 Minor 카테고리별 통계:")
        category_stats = df_result.groupby(['display_category_major', 'display_category_middle', 'display_category_minor']).agg({
            'key': 'count',
            'minor_total': 'first'
        }).rename(columns={'key': 'key_count', 'minor_total': 'product_count'}).reset_index()
        category_stats = category_stats.sort_values('product_count', ascending=False)
        
        print("\n상위 10개 Minor 카테고리:")
        for idx, row in category_stats.head(10).iterrows():
            print(f"   • [{row['display_category_major']}] > [{row['display_category_middle']}] > [{row['display_category_minor']}]: "
                  f"{row['product_count']}개 제품, {row['key_count']}개 key")
        
        # 파일로 저장 (탭 구분)
        output_file = 'category_minor_spec_analysis.txt'
        
        # 저장할 컬럼 순서 정의
        columns_order = [
            'display_category_major', 
            'display_category_middle',
            'display_category_minor',
            'key', 
            'product_count',
            'minor_total',
            'ratio_in_minor',
            'middle_total',
            'ratio_in_middle',
            'major_total', 
            'ratio_in_major',
            'unique_values_count',
            'unique_values'
        ]
        
        # 탭으로 구분된 텍스트 파일로 저장
        df_result[columns_order].to_csv(output_file, sep='\t', index=False, encoding='utf-8')
        
        print(f"\n💾 파일 저장 완료: {output_file}")
        print(f"   - 파일 크기: {os.path.getsize(output_file) / 1024:.2f} KB")
        print(f"   - 총 {len(df_result):,}개 행")
        print(f"   - 엑셀에서 열 때: 텍스트 가져오기 → 탭으로 구분")
        
        # 샘플 데이터 표시 (비율 포함)
        print(f"\n📋 데이터 샘플 (처음 10개 행):")
        display_cols = ['display_category_major', 'display_category_middle', 'display_category_minor', 'key', 
                       'product_count', 'ratio_in_minor', 'ratio_in_middle', 'ratio_in_major', 'unique_values_count']
        display(df_result[display_cols].head(10))
        
        # Minor 카테고리 내 비율이 높은 key들 분석
        print(f"\n📈 Minor 카테고리 내 비율이 100%인 Key (모든 제품이 가진 key):")
        perfect_ratio = df_result[df_result['ratio_in_minor'] == 100].sort_values(['minor_total', 'display_category_minor'], ascending=[False, True])
        for idx, row in perfect_ratio.head(15).iterrows():
            print(f"   • [{row['display_category_major']}] > [{row['display_category_middle']}] > [{row['display_category_minor']}] > '{row['key']}' "
                  f"({row['product_count']}/{row['minor_total']})")
        
        # Minor별 key 다양성 분석
        minor_diversity = df_result.groupby(['display_category_minor']).agg({
            'key': 'nunique',
            'minor_total': 'first'
        }).rename(columns={'key': 'unique_keys'}).reset_index()
        minor_diversity = minor_diversity.sort_values('unique_keys', ascending=False)
        
        print(f"\n🎯 Key 다양성이 높은 Minor 카테고리 Top 10:")
        for idx, row in minor_diversity.head(10).iterrows():
            print(f"   • {row['display_category_minor']}: {row['unique_keys']}개 고유 key ({row['minor_total']}개 제품)")
        
        # 특정 Minor 카테고리 상세 보기
        print(f"\n🔍 특정 Minor 카테고리 예시 (갤럭시 S):")
        galaxy_s_data = df_result[df_result['display_category_minor'] == '갤럭시 S']
        if not galaxy_s_data.empty:
            galaxy_s_data = galaxy_s_data.sort_values('ratio_in_minor', ascending=False)
            for idx, row in galaxy_s_data.head(5).iterrows():
                print(f"\n   Key: '{row['key']}'")
                print(f"   제품 수: {row['product_count']}/{row['minor_total']} (Minor: {row['ratio_in_minor']:.1f}%)")
                print(f"   Middle 내 비율: {row['ratio_in_middle']:.1f}%")
                print(f"   Major 내 비율: {row['ratio_in_major']:.1f}%")
                print(f"   고유값 개수: {row['unique_values_count']}개")
                # 값이 너무 길면 일부만 표시
                values_preview = row['unique_values']
                if len(values_preview) > 100:
                    values_preview = values_preview[:100] + '...'
                print(f"   Values: {values_preview}")
        
        # Minor 카테고리가 없는 경우 분석
        null_minor = df_result[df_result['display_category_minor'].isna() | (df_result['display_category_minor'] == '')]
        if not null_minor.empty:
            print(f"\n⚠️ Minor 카테고리가 없는 레코드: {len(null_minor)}개")
        
        # 추가로 Excel 형식으로도 저장 (선택사항)
        excel_file = 'category_minor_spec_analysis.xlsx'
        try:
            # unique_values가 너무 길면 Excel 셀 제한(32,767자)에 걸릴 수 있으므로 제한
            df_excel = df_result.copy()
            df_excel['unique_values'] = df_excel['unique_values'].apply(
                lambda x: x[:32000] + '...(truncated)' if len(x) > 32000 else x
            )
            df_excel[columns_order].to_excel(excel_file, index=False, engine='openpyxl')
            print(f"\n💾 Excel 파일도 저장됨: {excel_file}")
        except Exception as e:
            print(f"\n⚠️ Excel 저장 실패 (파일이 너무 큼): {e}")
            print(f"   → 텍스트 파일({output_file})을 사용하세요")
        
        print(f"\n✅ 분석 완료!")
        print(f"   - df_all_categories_minor_analysis: Minor까지 포함한 전체 분석 결과")
        
        return df_result
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        import traceback
        traceback.print_exc()
        return None
    finally:
        conn.close()

# Minor 카테고리까지 포함한 분석 실행
df_all_categories_minor_analysis = analyze_all_categories_with_minor()

# 파일이 제대로 생성되었는지 확인
if os.path.exists('category_minor_spec_analysis.txt'):
    print(f"\n✅ 파일 확인: category_minor_spec_analysis.txt")
    print(f"   파일 경로: {os.path.abspath('category_minor_spec_analysis.txt')}")
    
    # 파일 처음 몇 줄 미리보기
    with open('category_minor_spec_analysis.txt', 'r', encoding='utf-8') as f:
        lines = f.readlines()[:5]
        print(f"\n📄 파일 미리보기 (처음 5줄):")
        for i, line in enumerate(lines, 1):
            # 탭으로 구분된 필드 확인
            fields = line.strip().split('\t')
            if i == 1:  # 헤더
                print(f"   헤더: {len(fields)}개 컬럼")
                print(f"   컬럼 목록:")
                for j, col in enumerate(fields[:10]):  # 처음 10개 컬럼만
                    print(f"      {j+1}. {col}")
            else:
                if len(fields) >= 7:
                    print(f"   {i}: [{fields[2]}] '{fields[3]}' - "
                          f"{fields[6]}% in minor, {fields[8]}% in middle, {fields[10]}% in major")

In [ ]:
df_galaxy_smartphone_values.sort_values(['display_category_major','display_category_middle','key'])

In [ ]:
df_monitor_values.sort_values(['display_category_major','display_category_middle','key'])

In [ ]:
df_tv_values.sort_values(['display_category_major','display_category_middle','key'])

In [ ]:
# PC/주변기기	모니터	215
# 모바일	갤럭시 스마트폰	94
# TV	TV	175

result = df_grouped.groupby(['display_category_major', 'display_category_middle']).size().reset_index().rename(columns={0:'count'})
result

In [ ]:
result.sort_values(by=['count'],ascending=False)

In [1]:
# PC/주변기기	모니터	215
# 모바일	갤럭시 스마트폰	94
# TV	TV	175
import pandas as pd
pd.set_option('display.max_columns', None) ## 모든 열을 출력한다.
pd.set_option('display.max_rows', None) ## 모든 열을 출력한다.
 
df_grouped[df_grouped['display_category_middle'] == '갤럭시 스마트폰'].head(100)
# for idx, token in enumerate(df_grouped[df_grouped['display_category_middle'] == '갤럭시 스마트폰'].iterrows()):
#     print(f"{idx} : {token[1]}")

NameError: name 'df_grouped' is not defined

In [ ]:
/Users/toby/prog/kt/rubicon/data/kt_spec_validation_table_20251021.tsv